# Data wrangling

Notebook for doing some data wrangling and creating the plots for all the basins.

In [ ]:
import xarray as xr
import cartopy.crs as ccrs
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from matplotlib import ticker as plticker
import matplotlib.dates as mdates
from matplotlib.ticker import StrMethodFormatter, ScalarFormatter
from gha.hydro import calc_SPEI
import seaborn as sns
import geopandas as gpd
import pandas as pd
import re
import os
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
plt.rcParams['figure.figsize'] = (10, 7)
sns.set_theme()
fig_path = '/home/users/eholmgren/www_eholmgren/msc_thesis/plots/'

In [ ]:
def get_figsize(columnwidth, wf=0.5, hf=(5.**0.5-1.0)/2.0, ):
      """Parameters:
        - wf [float]:  width fraction in columnwidth units
        - hf [float]:  height fraction in columnwidth units.
                       Set by default to golden ratio.
        - columnwidth [float]: width of the column in latex. Get this from LaTeX 
                               using \showthe\columnwidth
      Returns:  [fig_width,fig_height]: that should be given to matplotlib
      """
      fig_width_pt = columnwidth*wf 
      inches_per_pt = 1.0/72.27               # Convert pt to inch
      fig_width = fig_width_pt*inches_per_pt  # width in inches
      fig_height = fig_width*hf      # height in inches
      return [fig_width, fig_height]

In [ ]:
basins_df = gpd.read_file('./data/glacier_basins.shp')
gcm_df = pd.read_csv('/home/www/oggm/cmip6/all_gcm_list.csv', index_col=0)

In [ ]:
basins_df

In [ ]:
gcms = gcm_df.gcm.unique()
gcms = np.delete(gcms, 10)
gcms

We want to read in each scenario for each basin. Keep them in a dict.

In [ ]:
# Base path where data is stored on the cluster.
base_path = '/home/users/eholmgren/work/gha_basins/'
# List with rcp scenarios
ssps = ['ssp126', 'ssp245', 'ssp370', 'ssp585']

In [ ]:
# We have a dual loop to load the data into a nested dict.
basin_dict = {}
for basin in basins_df.MRBID:
    gcm_dict = {}
    mrbid = str(basin)
    for gcm in gcms:
        scenario_dict = {}
        for ssp in ssps:
            # Create the path to the basin folder.
            path = os.path.join(base_path, mrbid) 
            # Name of the file.
            file = f'{mrbid}_discharge_proj_{gcm}_{ssp}.nc' 
            # Full path to the file.
            path = os.path.join(path, file)
            # Open the file with xarray.
            try:
                with xr.open_dataset(path, use_cftime=True) as ds:
                    # Last year is not good.
                    scenario_dict[ssp] = ds.isel(time=slice(0, -12))
            except FileNotFoundError:
                continue
            # Put the df in a dict.
            gcm_dict[gcm] = scenario_dict
    # Put the gcm dict in the basin dict.
    basin_dict[mrbid] = gcm_dict 

## Year of peak water

In [ ]:
df_peak_water = pd.DataFrame(columns=['mrbid', 'basin_name', 'ssp126-means', 'ssp245-means', 'ssp370-means',
                                      'ssp585-means'])


peak_water_dict = {}
for basin in basins_df.iloc:
    mrbid = str(basin.MRBID)
    basin_name = re.split(r'[\(\)]', basin.RIVER_BASI)[0].title()
    ssp_dict = {}
    for ssp in ssps:
        ens_list = []
        for gcm in gcms:
            try:
                df = basin_dict[mrbid][gcm][ssp]
                annual_runoff = df.glacier_runoff.isel(time=slice(0, -12)).groupby('time.year').sum()
                annual_runoff = annual_runoff.rolling(year=11, center=True, min_periods=1).mean()
                ens_list.append(annual_runoff)
            except KeyError:
                continue

        # When all is in the list. concat.
        ens_df = xr.concat(ens_list, dim='gcm')
        year_means = ens_df.isel(year=ens_df.argmax(dim='year')).year.values
        ssp_dict[ssp] = year_means
    #ssp_df = xr.concat(ssp_list, dim='ssp')
    peak_water_dict[mrbid] = ssp_dict
#peak_water_df = xr.concat(basin_list, dim='basin')
     


In [ ]:
# We want to put the peak water values in a large 
image_data = np.zeros((75, 4, 14))
for i, basin in enumerate(basins_df.iloc):
    mrbid = str(basin.MRBID)
    for j, ssp in enumerate(ssps):
        values = peak_water_dict[mrbid][ssp]
        values.sort()
        image_data[i, j, :len(values)] = values
                          
image_data
image_data = image_data.reshape(image_data.shape[:-2] + (-1,))

In [ ]:
df_peak_water.to_csv('/home/users/eholmgren/www_eholmgren/msc_thesis/peak_water_years.csv', sep=',')

### Peak water plot idea

In [ ]:
basin_names= [re.split(r'[\(\)]', name)[0].title() for name in basins_df.RIVER_BASI]

In [ ]:
import matplotlib.transforms as transform

In [ ]:
# Main figure
fig, ax = plt.subplots(figsize=(9/2.54, 28/2.54))
# color axes
divider = make_axes_locatable(ax)
cax = divider.append_axes('bottom', size='2%', pad=0.05)
# image data, mask bad values
im_data_masked = np.ma.masked_where(image_data == 0, image_data)
# Some utility vars.
xlen = data_im.shape[1]
ylen = data_im.shape[0]
#cmap
cmap = plt.get_cmap('viridis')
cmap.set_bad(color='lightgrey')
# The main colormesh
im = ax.pcolormesh(im_data_masked[::-1], cmap=cmap, vmin=2020., vmax=2100.,
                   edgecolors='face', lw=0)
# Add labels to the pixels. Median value.
for i in range(ylen):
    for j in range(4):
        # Median value
        ax.text(7+14*j, (74-i)+0.4, int(data_im[i, (j*14)+7]-2000),
                ha="center", va="center", color="w",
                fontsize=4, transform=ax.transData,
               )
       # ax.text(1+(14*j), (74-i)+0.4, int((data_im[i, 14*j]-2000)),
       #         ha="left", va="center", color="w",
       #         fontsize=5,
       #        )
       # ax.text(13+14*j, (74-i)+0.4, int(data_im[i, (j*12)+14]-2000),
       #         ha="right", va="center", color="w",
       #         fontsize=5, transform=ax.transData,
       #        )

# Ylabels and ticks
ax.set_yticks(np.arange(data.shape[0])[::-1]+0.5)
ax.set_yticklabels(basin_names)
# Xlabels and ticks. Loop cause I'm too lazy to deal with tickers.
ax.set_xticklabels([])
for i, ssp in enumerate(['SSP1-2.6', 'SSP2-4.5', 'SSP3-7.0', 'SSP5-8.5']):
    ax.text(7+i*14, 75.5, ssp, ha='center', va='center',
           transform=ax.transData,
           fontsize=5)
    
    
# Remove ticks
ax.tick_params('x', top=False, pad=-5)
# Change y tick padding.
ax.tick_params('y', pad=-4)
# Labelsize of yticks.
ax.tick_params('y', labelsize=7)
# Add the colorbar.
fig.colorbar(im, cax=cax, orientation='horizontal')
# Colorbar tick params.
cax.tick_params('both', labelsize=7, width=0.5, length=4)
cax.set_xlabel('Year', fontsize=7)
# Turn off grid.
ax.grid(None)

# Title
ax.set_title('Peak water ensemble estimations for 75 large scale basins', fontsize=7,
            x=0.5, y=1.02, transform=ax.transAxes, ha='center')
# Save figure.
fig.savefig(fig_path+'peak_years.pdf', bbox_inches='tight')

In [ ]:
patch.get_hatch()

## Plot prototype for ensemble

In [ ]:
fig, ax = plt.subplots()

basin = '2306'
# Loop over the dict.
max_values = []
max_years = []
for ssp in ssps:
    ens_list = []
    for gcm in gcms:
        try:
            df = basin_dict[basin][gcm][ssp]
            annual_runoff = df.glacier_runoff.isel(time=slice(0, -12)).groupby('time.year').sum()
            annual_runoff = annual_runoff.rolling(year=11, center=True, min_periods=1).mean()
            annual_runoff = (annual_runoff - annual_runoff.isel(year=0)) / annual_runoff.isel(year=0)
            ens_list.append(annual_runoff)
        except KeyError:
            continue
    
    
    # When all is in the list. concat.
    ens_df = xr.concat(ens_list, dim='gcm')
    q25 = ens_df.quantile(0.25, dim='gcm')
    q75 = ens_df.quantile(0.75, dim='gcm')
    ax.fill_between(q25.year, q25, q75, zorder=1, alpha=0.3)
    label = f'{ssp}: N={len(ens_df)}'
    ens_df.mean(dim='gcm').plot(ax=ax, label=label)
    max_value = ens_df.mean(dim='gcm').max().values
    max_values.append(max_value)
    max_year = ens_df.mean(dim='gcm').isel(year=ens_df.mean(dim='gcm').argmax()).year
    max_years.append(max_year)
    ymin, ymax = ax.get_ylim()
    
colors = ['C0', 'C1', 'C2', 'C3']
for value, year, color in zip(max_values, max_years, colors):
    ax.axvline(year, ymax=(value-ymin)/(ymax-ymin), zorder=3,
               ls=':', color=color, lw=2)
plt.legend(loc='best');

In [ ]:
ax._get_lines.prop_cycler

Lets plot everything just to have a look at it.

**This is the pure runoff data**

In [ ]:
basins_df.CONTINENT.unique()

## Europe

In [ ]:
continent = 'Europe'
basins = basins_df.loc[basins_df.CONTINENT == continent]

shift = 1
nbasins = len(basins)
ncols = 4
nrows = (nbasins // ncols) + shift
nplots = ncols*nrows
height_factor = 24/6
height = height_factor * nrows

fig, ax = plt.subplots(ncols=ncols, nrows=nrows, figsize=(15/2.54, height/2.54),
                       sharex=True, sharey=False)
labels = []
# font size
font_size = 7
# Plot all the basins
for i, basin in enumerate(basins.MRBID):
    #  Want to plot it in the last 75 subplots.
    idx = i + shift
    mrbid = str(basin)
    # Loop over the dicts.
    max_values = []
    max_years = []
    ymin = 0
    ymax = 0
    for ssp in ssps:
        ens_list = []
        for gcm in gcms:
            try:
                df = basin_dict[mrbid][gcm][ssp]
                annual_runoff = df.glacier_runoff.isel(time=slice(0, -12)).groupby('time.year').sum()
                annual_runoff = annual_runoff.rolling(year=11, center=True, min_periods=1).mean()
                annual_runoff_chg = (annual_runoff - annual_runoff.isel(year=0)) / annual_runoff.isel(year=0)
                annual_runoff_chg *= 100
                ens_list.append(annual_runoff_chg)
            except KeyError:
                continue

        # When all is in the list. concat.
        ens_df = xr.concat(ens_list, dim='gcm')
        q25 = ens_df.quantile(0.25, dim='gcm')
        q75 = ens_df.quantile(0.75, dim='gcm')
        ax.flat[idx].fill_between(q25.year, q25, q75, zorder=1, alpha=0.3)
        label = f'{ssp}\n N={len(ens_df)}'
        ens_df.mean(dim='gcm').plot(ax=ax.flat[idx], lw=0.8, label=label)
        # Stuff for vlines
        max_value = ens_df.mean(dim='gcm').max().values
        max_values.append(max_value)
        max_year = ens_df.mean(dim='gcm').isel(year=ens_df.mean(dim='gcm').argmax()).year
        max_years.append(max_year)
        ymin_t, ymax_t = ax.flat[idx].get_ylim()
        if ymin_t < ymin:
            ymin = ymin_t
        if ymax_t > ymax:
            ymax = ymax_t
    
        ax.flat[idx].set_ylabel('')
        ax.flat[idx].set_xlabel('')
        ax.flat[idx].tick_params(axis='y', labelsize=font_size, pad=-5)
        ax.flat[idx].tick_params(axis='x', labelsize=font_size, pad=-2)
        ax.flat[idx].xaxis.set_major_locator(mdates.AutoDateLocator(minticks=2, maxticks=5))

        # Grid
        ax.flat[idx].grid(which='both', linewidth=0.5)


        # Fix the years
        #ax.flat[idx].set_xlim(2020, 2100)

        # Get the basin name. 
        name = basins.iloc[i].RIVER_BASI
        # We don't need to plot the alternative names for now, so split it.
        name = re.split(r'[\(\)]', name)[0]
        if mrbid == '2910':
            name = name.split()[1]
        if mrbid == '6101':
            name = name.split()[0]
        ax.flat[idx].set_title(f'{name}', size=font_size, pad=1.1)
        ax.flat[idx].tick_params('both')
    # Plot vlines
    colors = ['C0', 'C1', 'C2', 'C3']
    for value, year, color in zip(max_values, max_years, colors):
        ax.flat[idx].axvline(year, ymax=(value-ymin)/(ymax-ymin), zorder=3,
                   ls=':', color=color, lw=0.7)

# Make the first plots invisible.
for i in range(shift):
    ax.flat[i].set_visible(False)
for i in range(nplots - (nplots - nbasins - 1), nplots):
    ax.flat[i].set_visible(False)

# Make a legend.
handles, labels = ax.flat[5].get_legend_handles_labels()
fig.legend(handles, labels, ncol=1, bbox_to_anchor=(0.2, 0.8), fontsize=font_size,
           columnspacing=0.5, handlelength=1, loc='center')
# Figure title
# Do it in inkscape
# Common x/y labels
fig.text(0.5, 0.07, 'Year', ha='center', fontsize=font_size)
fig.text(0.07, 0.5, 'Relative change (%)', va='center', rotation='vertical', fontsize=font_size)
fig.text(0.5, 0.91, f'Glacier runoff projection ensembles for {continent}', ha='center', fontsize=font_size)


#fig.tight_layout()
plt.savefig(fig_path+f'peak_water_europe.pdf', bbox_inches='tight')

## Asia / SW pacific

In [ ]:
continent = 'Asia'
basins = basins_df.loc[(basins_df.CONTINENT.eq('Asia')) | (basins_df.CONTINENT.eq('South-West Pacific'))]

shift = 0
nbasins = len(basins)
ncols = 4
nrows = (nbasins // ncols) + shift
nplots = ncols*nrows
height_factor = 24/6
height = height_factor * nrows

fig, ax = plt.subplots(ncols=ncols, nrows=nrows, figsize=(15/2.54, height/2.54),
                       sharex=True, sharey=False)
labels = []
# font size
font_size = 7
# Plot all the basins
for i, basin in enumerate(basins.MRBID):
    #  Want to plot it in the last 75 subplots.
    idx = i + shift
    mrbid = str(basin)
    # Loop over the dicts.
    for ssp in ssps:
        ens_list = []
        for gcm in gcms:
            try:
                df = basin_dict[mrbid][gcm][ssp]
                annual_runoff = df.glacier_runoff.isel(time=slice(0, -12)).groupby('time.year').sum()
                annual_runoff = annual_runoff.rolling(year=11, center=True, min_periods=1).mean()
                annual_runoff_chg = (annual_runoff - annual_runoff.isel(year=0)) / annual_runoff.isel(year=0)
                annual_runoff_chg *= 100
                ens_list.append(annual_runoff_chg)
            except KeyError:
                continue

        # When all is in the list. concat.
        ens_df = xr.concat(ens_list, dim='time')
        q25 = ens_df.quantile(0.25, dim='time')
        q75 = ens_df.quantile(0.75, dim='time')
        ax.flat[idx].fill_between(q25.year, q25, q75, zorder=1, alpha=0.3)
        label = f'{ssp}\n N={len(ens_df)}'
        ens_df.mean(dim='time').plot(ax=ax.flat[idx], lw=0.8, label=label)
        ax.flat[idx].set_ylabel('')
        ax.flat[idx].set_xlabel('')
        ax.flat[idx].tick_params(axis='y', labelsize=font_size, pad=-5)
        ax.flat[idx].tick_params(axis='x', labelsize=font_size)

        # Grid
        ax.flat[idx].grid(which='both', linewidth=0.5)


        # Fix the years
        #ax.flat[idx].set_xlim(2020, 2100)

        # Get the basin name. 
        name = basins.iloc[i].RIVER_BASI
        # We don't need to plot the alternative names for now, so split it.
        name = re.split(r'[\(\)]', name)[0]
        if mrbid == '2910':
            name = name.split()[1]
        if mrbid == '6101':
            name = name.split()[0]
        ax.flat[idx].set_title(f'{name}', size=font_size, pad=1.1)
        ax.flat[idx].tick_params('both')

# Make the first plots invisible.
for i in range(shift):
    ax.flat[i].set_visible(False)
for i in range(nplots - (nplots - nbasins - 1), nplots):
    ax.flat[i].set_visible(False)

# Make a legend.
handles, labels = ax.flat[5].get_legend_handles_labels()
fig.legend(handles, labels, ncol=4, bbox_to_anchor=(0.5, 0.92), fontsize=font_size,
           columnspacing=0.5, handlelength=1, loc='center')
# Figure title
# Do it in inkscape
# Common x/y labels
fig.text(0.5, 0.08, 'Year', ha='center', fontsize=font_size)
fig.text(0.07, 0.5, 'Relative change (%)', va='center', rotation='vertical', fontsize=font_size)
fig.text(0.5, 0.95, f'Glacier runoff projection ensembles for Asia and New Zealand', ha='center', fontsize=font_size)


#fig.tight_layout()
plt.savefig(fig_path+f'peak_water_asia_nz.pdf', bbox_inches='tight')

## South America

In [ ]:
continent = 'South America'
basins = basins_df.loc[(basins_df.CONTINENT == continent)]

shift = 0
nbasins = len(basins)
ncols = 4
nrows = (nbasins // ncols) + shift
nplots = ncols*nrows
height_factor = 24/6
height = height_factor * nrows

fig, ax = plt.subplots(ncols=ncols, nrows=nrows, figsize=(15/2.54, height/2.54),
                       sharex=True, sharey=False)
labels = []
# font size
font_size = 7
# Plot all the basins
for i, basin in enumerate(basins.MRBID):
    #  Want to plot it in the last 75 subplots.
    idx = i + shift
    mrbid = str(basin)
    # Loop over the dicts.
    for ssp in ssps:
        ens_list = []
        for gcm in gcms:
            try:
                df = basin_dict[mrbid][gcm][ssp]
                annual_runoff = df.glacier_runoff.isel(time=slice(0, -12)).groupby('time.year').sum()
                annual_runoff = annual_runoff.rolling(year=11, center=True, min_periods=1).mean()
                annual_runoff_chg = (annual_runoff - annual_runoff.isel(year=0)) / annual_runoff.isel(year=0)
                annual_runoff_chg *= 100
                ens_list.append(annual_runoff_chg)
            except KeyError:
                continue

        # When all is in the list. concat.
        ens_df = xr.concat(ens_list, dim='time')
        q25 = ens_df.quantile(0.25, dim='time')
        q75 = ens_df.quantile(0.75, dim='time')
        ax.flat[idx].fill_between(q25.year, q25, q75, zorder=1, alpha=0.3)
        label = f'{ssp}\n N={len(ens_df)}'
        ens_df.mean(dim='time').plot(ax=ax.flat[idx], lw=0.8, label=label)
        ax.flat[idx].set_ylabel('')
        ax.flat[idx].set_xlabel('')
        ax.flat[idx].tick_params(axis='y', labelsize=font_size, pad=-5)
        ax.flat[idx].tick_params(axis='x', labelsize=font_size)

        # Grid
        ax.flat[idx].grid(which='both', linewidth=0.5)


        # Fix the years
        #ax.flat[idx].set_xlim(2020, 2100)

        # Get the basin name. 
        name = basins.iloc[i].RIVER_BASI
        # We don't need to plot the alternative names for now, so split it.
        name = re.split(r'[\(\)]', name)[0]
        if mrbid == '2910':
            name = name.split()[1]
        if mrbid == '6101':
            name = name.split()[0]
        ax.flat[idx].set_title(f'{name}', size=font_size, pad=1.1)
        ax.flat[idx].tick_params('both')

# Make the first plots invisible.
for i in range(shift):
    ax.flat[i].set_visible(False)
for i in range(nplots - (nplots - nbasins - 1), nplots):
    ax.flat[i].set_visible(False)

# Make a legend.
handles, labels = ax.flat[5].get_legend_handles_labels()
fig.legend(handles, labels, ncol=4, bbox_to_anchor=(0.5, 0.92), fontsize=font_size,
           columnspacing=0.5, handlelength=1, loc='center')
# Figure title
# Do it in inkscape
# Common x/y labels
fig.text(0.5, 0.09, 'Year', ha='center', fontsize=font_size)
fig.text(0.07, 0.5, 'Relative change (%)', va='center', rotation='vertical', fontsize=font_size)
fig.text(0.5, 0.95, f'Glacier runoff projection ensembles for South America', ha='center', fontsize=font_size)


#fig.tight_layout()
plt.savefig(fig_path+f'peak_water_s_america.pdf', bbox_inches='tight')

## N. America

In [ ]:
basins_df.CONTINENT.unique()

In [ ]:
continent = 'North America, Central America and the Caribbean'
basins = basins_df.loc[(basins_df.CONTINENT == continent)]

shift = 0
nbasins = len(basins)
ncols = 4
nrows = (nbasins // ncols) + shift
nplots = ncols*nrows
height_factor = 24/6
height = height_factor * nrows

fig, ax = plt.subplots(ncols=ncols, nrows=nrows, figsize=(15/2.54, height/2.54),
                       sharex=True, sharey=False)
labels = []
# font size
font_size = 7
# Plot all the basins
for i, basin in enumerate(basins.MRBID):
    #  Want to plot it in the last 75 subplots.
    idx = i + shift
    mrbid = str(basin)
    # Loop over the dicts.
    max_values = []
    max_years = []
    ymin = 0
    ymax = 0
    for ssp in ssps:
        ens_list = []
        for gcm in gcms:
            try:
                df = basin_dict[mrbid][gcm][ssp]
                annual_runoff = df.glacier_runoff.isel(time=slice(0, -12)).groupby('time.year').sum()
                annual_runoff = annual_runoff.rolling(year=11, center=True, min_periods=1).mean()
                annual_runoff_chg = (annual_runoff - annual_runoff.isel(year=0)) / annual_runoff.isel(year=0)
                annual_runoff_chg *= 100
                ens_list.append(annual_runoff_chg)
            except KeyError:
                continue

        # When all is in the list. concat.
        ens_df = xr.concat(ens_list, dim='gcm')
        q25 = ens_df.quantile(0.25, dim='gcm')
        q75 = ens_df.quantile(0.75, dim='gcm')
        ax.flat[idx].fill_between(q25.year, q25, q75, zorder=1, alpha=0.3)
        # Stuff for vlines
        max_value = ens_df.mean(dim='gcm').max().values
        max_values.append(max_value)
        max_year = ens_df.mean(dim='gcm').isel(year=ens_df.mean(dim='gcm').argmax()).year
        max_years.append(max_year)
        ymin_t, ymax_t = ax.flat[idx].get_ylim()
        if ymin_t < ymin:
            ymin = ymin_t
        if ymax_t > ymax:
            ymax = ymax_t
    
        label = f'{ssp}\n N={len(ens_df)}'
        ens_df.mean(dim='gcm').plot(ax=ax.flat[idx], lw=0.8, label=label)
        ax.flat[idx].set_ylabel('')
        ax.flat[idx].set_xlabel('')
        ax.flat[idx].tick_params(axis='y', labelsize=font_size, pad=-5)
        ax.flat[idx].tick_params(axis='x', labelsize=font_size, pad=-2)
        ax.flat[idx].xaxis.set_major_locator(mdates.AutoDateLocator(minticks=2, maxticks=5))

        # Grid
        ax.flat[idx].grid(which='both', linewidth=0.5)


        # Fix the years
        #ax.flat[idx].set_xlim(2020, 2100)

        # Get the basin name. 
        name = basins.iloc[i].RIVER_BASI
        # We don't need to plot the alternative names for now, so split it.
        name = re.split(r'[\(\)]', name)[0]
        if mrbid == '2910':
            name = name.split()[1]
        if mrbid == '6101':
            name = name.split()[0]
        ax.flat[idx].set_title(f'{name}', size=font_size, pad=1.1)
        ax.flat[idx].tick_params('both')
    # Plot vlines
    colors = ['C0', 'C1', 'C2', 'C3']
    for value, year, color in zip(max_values, max_years, colors):
        ax.flat[idx].axvline(year, ymax=(value-ymin)/(ymax-ymin), zorder=3,
                   ls=':', color=color, lw=0.7)
# Make the first plots invisible.
for i in range(shift):
    ax.flat[i].set_visible(False)
for i in range(nplots - (nplots - nbasins - 1), nplots):
    ax.flat[i].set_visible(False)

# Make a legend.
handles, labels = ax.flat[5].get_legend_handles_labels()
fig.legend(handles, labels, ncol=4, bbox_to_anchor=(0.5, 0.92), fontsize=font_size,
           columnspacing=0.5, handlelength=1, loc='center')
# Figure title
# Do it in inkscape
# Common x/y labels
fig.text(0.5, 0.08, 'Year', ha='center', fontsize=font_size)
fig.text(0.07, 0.5, 'Relative change (%)', va='center', rotation='vertical', fontsize=font_size)
fig.text(0.5, 0.95, f'Glacier runoff projection ensembles for North America', ha='center', fontsize=font_size)


#fig.tight_layout()
plt.savefig(fig_path+f'peak_water_n_america.pdf', bbox_inches='tight')

### N. America presentation

In [ ]:
continent = 'North America, Central America and the Caribbean'
basins = basins_df.loc[(basins_df.CONTINENT == continent)]

shift = 0
nbasins = len(basins)
ncols = 4
nrows = (nbasins // ncols) + shift
nplots = ncols*nrows
height_factor = 24/6
height = 9.8

fig, ax = plt.subplots(ncols=ncols, nrows=nrows, figsize=(15/2.54, height/2.54),
                       sharex=True, sharey=False)
labels = []
# font size
font_size = 7
# Plot all the basins
for i, basin in enumerate(basins.MRBID):
    #  Want to plot it in the last 75 subplots.
    idx = i + shift
    mrbid = str(basin)
    # Loop over the dicts.
    max_values = []
    max_years = []
    ymin = 0
    ymax = 0
    for ssp in ssps:
        ens_list = []
        for gcm in gcms:
            try:
                df = basin_dict[mrbid][gcm][ssp]
                annual_runoff = df.glacier_runoff.isel(time=slice(0, -12)).groupby('time.year').sum()
                annual_runoff = annual_runoff.rolling(year=11, center=True, min_periods=1).mean()
                annual_runoff_chg = (annual_runoff - annual_runoff.isel(year=0)) / annual_runoff.isel(year=0)
                annual_runoff_chg *= 100
                ens_list.append(annual_runoff_chg)
            except KeyError:
                continue

        # When all is in the list. concat.
        ens_df = xr.concat(ens_list, dim='gcm')
        q25 = ens_df.quantile(0.25, dim='gcm')
        q75 = ens_df.quantile(0.75, dim='gcm')
        ax.flat[idx].fill_between(q25.year, q25, q75, zorder=1, alpha=0.3)
        # Stuff for vlines
        max_value = ens_df.mean(dim='gcm').max().values
        max_values.append(max_value)
        max_year = ens_df.mean(dim='gcm').isel(year=ens_df.mean(dim='gcm').argmax()).year
        max_years.append(max_year)
        ymin_t, ymax_t = ax.flat[idx].get_ylim()
        if ymin_t < ymin:
            ymin = ymin_t
        if ymax_t > ymax:
            ymax = ymax_t
    
        label = f'{ssp} (N={len(ens_df)})'
        ens_df.mean(dim='gcm').plot(ax=ax.flat[idx], lw=0.8, label=label)
        ax.flat[idx].set_ylabel('')
        ax.flat[idx].set_xlabel('')
        ax.flat[idx].tick_params(axis='y', labelsize=font_size, pad=-5)
        ax.flat[idx].tick_params(axis='x', labelsize=font_size, pad=-2)
        ax.flat[idx].xaxis.set_major_locator(mdates.AutoDateLocator(minticks=2, maxticks=5))

        # Grid
        ax.flat[idx].grid(which='both', linewidth=0.5)


        # Fix the years
        #ax.flat[idx].set_xlim(2020, 2100)

        # Get the basin name. 
        name = basins.iloc[i].RIVER_BASI
        # We don't need to plot the alternative names for now, so split it.
        name = re.split(r'[\(\)]', name)[0]
        if mrbid == '2910':
            name = name.split()[1]
        if mrbid == '6101':
            name = name.split()[0]
        ax.flat[idx].set_title(f'{name}', size=font_size, pad=1.1)
        ax.flat[idx].tick_params('both')
    # Plot vlines
    colors = ['C0', 'C1', 'C2', 'C3']
    for value, year, color in zip(max_values, max_years, colors):
        ax.flat[idx].axvline(year, ymax=(value-ymin)/(ymax-ymin), zorder=3,
                   ls=':', color=color, lw=0.7)
# Make the first plots invisible.
for i in range(shift):
    ax.flat[i].set_visible(False)
for i in range(nplots - (nplots - nbasins - 1), nplots):
    ax.flat[i].set_visible(False)

# Make a legend.
handles, labels = ax.flat[5].get_legend_handles_labels()
fig.legend(handles, labels, ncol=4, bbox_to_anchor=(0.5, 0.94), fontsize=font_size,
           columnspacing=0.5, handlelength=1, loc='center')
# Figure title
# Do it in inkscape
# Common x/y labels
fig.text(0.5, 0.06, 'Year', ha='center', fontsize=font_size)
fig.text(0.07, 0.5, 'Relative change (%)', va='center', rotation='vertical', fontsize=font_size)
fig.text(0.5, 0.98, f'Glacier runoff projection ensembles for North America', ha='center', fontsize=font_size)


#fig.tight_layout()
plt.savefig(fig_path+f'peak_water_n_america_pres.pdf', bbox_inches='tight', facecolor='none')

In [ ]:
basins_df.CONTINENT.unique()

# SPEI

In [ ]:
# We have a dual loop to load the data into a nested dict.
SPEI_dict = {}
for basin in basins_df.MRBID:
    gcm_dict = {}
    mrbid = str(basin)
    for gcm in gcms:
        # Create the path to the basin folder.
        path = os.path.join(base_path, mrbid) 
        # Name of the file.
        file = f'{mrbid}_SPEI_{gcm}_False_wref.nc' 
        # Full path to the file.
        path = os.path.join(path, file)
        # Open the file with xarray.
        with xr.open_dataset(path, use_cftime=True) as ds:
            gcm_dict[gcm] = ds
    # Put the scenario dict in the basin dict.
    SPEI_dict[mrbid] = gcm_dict 

In [ ]:
def plot_spei(rcp, window=None):
    '''Plot SPEI with and without glacier runoff for all basins for a given
    rcp scenario.
    
    Args:
    -----
    rcp: string
        I.e 'rcp26' for the rcp 2.6 scenario.
    window: Int
        Number of months in the rolling windo.
    '''
    
    fig, ax = plt.subplots(ncols=8, nrows=10, figsize=get_figsize(307, wf=4), sharex=True)
    # Plot all the basins
    for i, basin in enumerate(basins_df.MRBID):
        #  Want to plot it in the last 75 subplots.
        idx = i + 5
        mrbid = str(basin)
        # Plot all SPEI and adjusted SPEI
        # Get the SPEI
        # Adjusted
        SPEI_adj = SPEI_dict[mrbid][rcp].sel(time=slice('2019', '2100'))
        # if we want to smooth it.
        if window:
            SPEI_adj = SPEI_adj.SPEI_adj.rolling(time=window).mean()
        else:
            SPEI_adj = SPEI_adj.SPEI_adj
        SPEI_adj.plot(label=f'W. glaciers {rcp}', ax=ax.flat[idx])
        # Non-adjusted
        SPEI = SPEI_dict[mrbid][rcp].sel(time=slice('2019', '2100'))
        # If we want to smooth it.
        if window:
            SPEI = SPEI.SPEI.rolling(time=window).mean()
        else:
            SPEI = SPEI.SPEI
        SPEI.plot(label=f'W.o. glaciers {rcp}', ax=ax.flat[idx])
        # Title of each subplot.
        ax.flat[idx].set_title(f'{basins_df.iloc[i].RIVER_BASI}')
        # For Tornealven we need to shorten the title a bit.
        if i == 74:
            ax.flat[idx].set_title(f'{basins_df.iloc[i].RIVER_BASI[:10]}')
        # Remove the ylabel for all plots.
        ax.flat[idx].set_ylabel('')
        # Put add them back to the left-most plots.
        if not i%8:
            ax.flat[i].set_ylabel('SPEI')
        # And the first plot.
        # ax.flat[idx].set_xlabel('')
        if i == 5:
            ax.flat[i].set_ylabel('SPEI') 
        # styling
        ax.flat[idx].set_facecolor('0.9')
        ax.flat[idx].grid('True', axis='both', color='white', linewidth=0.5)
        ax.flat[idx].set_axisbelow(True)
        ax.flat[idx].spines['right'].set_visible(False)
        ax.flat[idx].spines['left'].set_visible(False)
        ax.flat[idx].spines['top'].set_visible(False)
        ax.flat[idx].spines['bottom'].set_visible(False)
        ax.flat[idx].yaxis.set_ticks_position('none')
        ax.flat[idx].xaxis.set_ticks_position('none')
    # Make the first five plots invisible.
    for i in range(5):
        ax.flat[i].set_visible(False)
    # Make a legend.
    handles, labels = ax.flat[5].get_legend_handles_labels()
    fig.legend(handles, labels, bbox_to_anchor=(0.5, 0.95), loc='center left', title='Rcp scenarios')
    # Add window information
    fig.suptitle(f'Length of rolling window: {window} months', x = 0.5, y = 0.99, ha='left')
    # Figure title
    # Do it in inkscape

    # Work with the margins    
    #plt.subplots_adjust(wspace=0.4, hspace=0.9)
    fig.tight_layout()

In [ ]:
plot_spei('rcp26', window=11*12)

In [ ]:
plot_spei('rcp45', window=11*12)

In [ ]:
plot_spei('rcp60', window=11*12)

In [ ]:
plot_spei('rcp85', window=11*12)

### Individual basins

In [ ]:
gcms

In [ ]:
def get_spei_ensemble(mrbid, ssp):
    
    spei_list = []
    spei_adj_list = []
    # Ensemble
    # try to conform all the series to the same calendar with use of one master idx.
    # Some of the gcms are on 16th instead of the 15th
    idx_master = SPEI_dict[mrbid][gcms[0]][f'SPEI_{ssp}'].sel(time=slice('2019', '2100')).indexes['time'].to_datetimeindex()
    for gcm in gcms:
        try:
            spei = SPEI_dict[mrbid][gcm][f'SPEI_{ssp}'].sel(time=slice('2019', '2100'))
            idx = spei.indexes['time'].to_datetimeindex()
            spei['time'] = idx 
            spei = spei.reindex({'time': idx_master}, method='nearest')
            spei_list.append(spei)
            spei_adj = SPEI_dict[mrbid][gcm][f'SPEI_adj_{ssp}'].sel(time=slice('2019', '2100'))
            idx = spei_adj.indexes['time'].to_datetimeindex()
            spei_adj['time'] = idx 
            spei_adj = spei_adj.reindex({'time': idx_master}, method='nearest')
            spei_adj_list.append(spei_adj)
        except KeyError:
            pass

    # Create the SPEI ensembles
    spei_ens = xr.concat(spei_list, dim='gcm')
    spei_adj_ens = xr.concat(spei_adj_list, dim='gcm')
    
    return spei_ens, spei_adj_ens

In [ ]:
spei_ens, spei_adj_ens = get_spei_ensemble('2103', 'ssp126')

In [ ]:
q25 = spei_ens.quantile(0.25, dim='gcm')
q25.time

In [ ]:
def plot_spei_basin(ssp, mrbid=None, basin_idx=None, window=None, nofac=False, savefig=False):
    
    if basin_idx is not None:
        basin = basins_df.iloc[basin_idx]
        mrbid = str(basin.MRBID)
    elif mrbid is not None:
        basin = basins_df.loc[basins_df.MRBID==mrbid].iloc[0]
        mrbid = str(basin.MRBID)
    if mrbid is None and basin_idx is None:
        print('Please provide mrbid or index')
        
    # Get the ensembles.
    spei_ens, spei_adj_ens = get_spei_ensemble(mrbid, ssp)
    # The plot
    font_size = 7
    fig, ax = plt.subplots(figsize=(15/2.54, 7/2.54))
    if not window:
        spei_ens.mean(dim='gcm').plot(ax=ax, lw=1, label='SPEI w.o glacier runoff', c='C1')
        spei_adj_ens.mean(dim='gcm').plot(ax=ax, lw=1, label='SPEI w. glacier runoff', c='C0')
    else:
        # w.o. glacier
        spei_ens.mean(dim='gcm').rolling(time=window).mean().plot(ax=ax, lw=1, label='SPEI w.o glacier runoff', c='C1')
        q25 = spei_ens.quantile(q=0.25, dim='gcm').rolling(time=window).mean()
        q75 = spei_ens.quantile(q=0.75, dim='gcm').rolling(time=window).mean()
        ax.fill_between(spei_ens.time.values, q25, q75, alpha=0.5, color='C1')
        # w. glacier
        spei_adj_ens.mean(dim='gcm').rolling(time=window).mean().plot(ax=ax, lw=1, label='SPEI w. glacier runoff', c='C0')
        q25 = spei_adj_ens.quantile(q=0.25, dim='gcm').rolling(time=window).mean()
        q75 = spei_adj_ens.quantile(q=0.75, dim='gcm').rolling(time=window).mean()
        ax.fill_between(spei_adj_ens.time.values, q25, q75, alpha=0.5, color='C0')
    
    # x and y labels
    ax.set_ylabel('SPEI', fontsize=font_size, labelpad=-1)
    ax.set_xlabel('Time', fontsize=font_size, labelpad=-1)

    # Some tick adjustments
    ax.tick_params('both', labelsize=font_size, pad=-4)
    plt.title(f'{basin.RIVER_BASI} {ssp}', fontsize=font_size)
    plt.legend(fontsize=font_size);

    name = basin.RIVER_BASI
    name = name.replace(' ', '_')
    if savefig:
        fig_name = f'{name}_{ssp}.pdf'
        path = os.path.join(fig_path, fig_name)
        fig.savefig(path, bbox_inches='tight')

In [ ]:
plot_spei_basin('ssp370', basin_idx=24, window=12*11, savefig=True)

In [ ]:
plot_spei_basin('ssp370', mrbid=2309, window=12*12, savefig=True)

In [ ]:
basin = basins_df.iloc[4]
mrbid = str(basin.MRBID)
ssp = 'ssp370'
# Get the ensembles
spei_ens, spei_adj_ens = get_spei_ensemble(mrbid, ssp)

# spei_ens = spei_ens.sel(time=spei_ens['time.season'] == 'JJA')
# spei_adj_ens = spei_adj_ens.sel(time=spei_adj_ens['time.season'] == 'JJA')

font_size = 7
fig, ax = plt.subplots(figsize=(5/2.54, 5/2.54))

# Add the mean
v1 = ax.violinplot(spei_ens.mean(dim='gcm'), showextrema=False, positions=[0])
v2 = ax.violinplot(spei_adj_ens.mean(dim='gcm'), showextrema=False, positions=[0])

for pc in v1['bodies']:
    # get the center
    m = np.mean(pc.get_paths()[0].vertices[:, 0])
    # modify the paths to not go further right than the center
    pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
    pc.set_facecolor('C1')
    pc.set_edgecolor('k')
    pc.set_alpha(0.5)
    pc.set_zorder(3)
for pc in v2['bodies']:
    # get the center
    m = np.mean(pc.get_paths()[0].vertices[:, 0])
    # modify the paths to not go further right than the center
    pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
    pc.set_facecolor('C0')
    pc.set_edgecolor('k')
    pc.set_alpha(0.5)
    pc.set_zorder(3)
    
# Add the 25th quantile 
v3 = ax.violinplot(spei_ens.quantile(q=0.25, dim='gcm'), showextrema=False, positions=[0.2])
v4 = ax.violinplot(spei_adj_ens.quantile(q=0.25, dim='gcm'), showextrema=False, positions=[0.2])
for pc in v3['bodies']:
    # get the center
    m = np.mean(pc.get_paths()[0].vertices[:, 0])
    # modify the paths to not go further right than the center
    pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
    pc.set_facecolor('C1')
    pc.set_edgecolor('k')
    pc.set_linewidth(1)
    pc.set_alpha(0.5)
    pc.set_zorder(2.5)
for pc in v4['bodies']:
    # get the center
    m = np.mean(pc.get_paths()[0].vertices[:, 0])
    # modify the paths to not go further right than the center
    pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
    pc.set_facecolor('C0')
    pc.set_edgecolor('k')
    pc.set_linewidth(1)
    pc.set_alpha(0.5)
    pc.set_zorder(2.5)
    
# Add the 75th quantile 
v3 = ax.violinplot(spei_ens.quantile(q=0.75, dim='gcm'), showextrema=False, positions=[0.4])
v4 = ax.violinplot(spei_adj_ens.quantile(q=0.75, dim='gcm'), showextrema=False, positions=[0.4])
for pc in v3['bodies']:
    # get the center
    m = np.mean(pc.get_paths()[0].vertices[:, 0])
    # modify the paths to not go further right than the center
    pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
    pc.set_facecolor('C1')
    pc.set_edgecolor('k')
    pc.set_linewidth(1)
    pc.set_alpha(0.5)
    pc.set_zorder(2)
for pc in v4['bodies']:
    # get the center
    m = np.mean(pc.get_paths()[0].vertices[:, 0])
    # modify the paths to not go further right than the center
    pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
    pc.set_facecolor('C0')
    pc.set_edgecolor('k')
    pc.set_linewidth(1)
    pc.set_alpha(0.5)
    pc.set_zorder(2)
# Extra stuff
ind = 1.0
offset = 0.02
# q1, median, q3 = spei.quantile([0.25, 0.5, 0.75]).values
# ax.vlines(ind+offset, q1, q3, color='C0', lw=5)
# q1, median, q3 = spei_adj.quantile([0.25, 0.5, 0.75]).values
# ax.vlines(ind-offset, q1, q3, color='C1', lw=5)

# Fix the xaxis
ax.set_xlim(0, 0.7)
ax.tick_params('both', labelsize=font_size)
plt.ylabel('SPEI', fontsize=font_size)
# Legend stuff
patches = [mpatches.Patch(color='C0', alpha=0.5),
           mpatches.Patch(color='C1', alpha=0.5)]
labels = ['W.o. glaciers',
          'W. glaciers']
# plt.legend(patches, labels)
plt.title(f'SPEI climatology {basin.RIVER_BASI}', fontsize=font_size);
plt.savefig(fig_path+'clim_test.pdf', bbox_inches='tight')

In [ ]:
def plot_spei_climatology(rcp, nofac=False, season=None, savefig=False, lat_sort=False):
    '''Plot the SPEI climatology (violins) of all basins for a certain rcp
    scenario.
    
    Args:
    -----
    rcp: str
        I.e. 'rcp26'
    season: str
        Choose a season to plot. 'winter' or 'summer'.
    '''
    selection_dict = {'summer': {True: 'JJA', False: 'DJF'},
                      'winter': {True: 'DJF', False: 'JJA'}}
    # Create the figure
    fig, ax = plt.subplots(ncols=11, nrows=7, figsize=get_figsize(307, wf=4.2),
                           sharex=True, sharey=False)
    c0 = 'C1'
    c1 = 'C0'
    c2 = 'C2'
    alpha = 0.5
    basins = basins_df
    basins['lat'] = basins['geometry'].centroid.y
    if lat_sort:
        basins['lat'] = basins['geometry'].centroid.y
        basins = basins.sort_values('lat', ascending=False)
        
    for i, basin in enumerate(basins.MRBID):
        shift = 2
        idx = i + shift
        mrbid = str(basin)
        spei = SPEI_dict[mrbid][rcp].sel(time=slice('2019', '2100')).SPEI
        spei_adj = SPEI_dict[mrbid][rcp].sel(time=slice('2019', '2100')).SPEI_adj
        if nofac:
            spei_adj_nofac = SPEI_dict[mrbid][rcp].sel(time=slice('2019', '2100')).SPEI_adj_nofac
        
        if season is not None:
            # Check wether we are in NH (true) or SH (False)
            hemisphere = basins.iloc[i]['lat'] > 0
            # Select the season.
            selection = selection_dict[season][hemisphere]
            spei = spei.sel(time=(spei['time.season']==selection))
            spei_adj = spei_adj.sel(time=(spei_adj['time.season']==selection))
            if nofac:
                spei_adj_nofac = spei_adj_nofac.sel(time=(spei_adj['time.season']==selection))

        v1 = ax.flat[idx].violinplot(spei, showextrema=False)
        v2 = ax.flat[idx].violinplot(spei_adj, showextrema=False)
        if nofac:
            v3 = ax.flat[idx].violinplot(spei_adj_nofac, showextrema=False)
            
        for pc in v1['bodies']:
            # get the center
            # m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            # pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], -np.inf, m)
            pc.set_facecolor(c0)
            pc.set_edgecolor('k')
            pc.set_alpha(alpha)
        for pc in v2['bodies']:
            # get the center
            # m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            # pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor(c1)
            pc.set_edgecolor('k')
            pc.set_alpha(alpha)

        if nofac:
            for pc in v3['bodies']:
                # get the center
                # m = np.mean(pc.get_paths()[0].vertices[:, 0])
                # modify the paths to not go further right than the center
                # pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
                pc.set_facecolor(c2)
                pc.set_edgecolor('k')
                pc.set_alpha(alpha)
        # Extra stuff
       #  ind = 1.0
       #  offset = 0.02
       #  q1, median, q3 = spei.quantile([0.25, 0.5, 0.75]).values
       #  # Median, kind of unnecessary to plot since it should be the same
       #  # for all by design of the index.
       #  # ax.flat[idx].scatter(ind+offset, median, c='k', zorder=3, s=20)
       #  ax.flat[idx].vlines(ind+offset, q1, q3, color=c0, lw=5, zorder=3)
       #  q1, median, q3 = spei_adj.quantile([0.25, 0.5, 0.75]).values
       #  # ax.flat[idx].scatter(ind-offset, median, c='k', zorder=3, s=20)
       #  ax.flat[idx].vlines(ind-offset, q1, q3, color=c1, lw=5, zorder=3)

        # Get the basin name. 
        name = basins.iloc[i].RIVER_BASI
        # We don't need to plot the alternative names for now, so split it.
        name = re.split(r'[\(\)]', name)[0]
        if mrbid == '2910':
            name = name.split()[1]
        if mrbid == '6101':
            name = name.split()[0]
        ax.flat[idx].set_title(f'{name}', color='white')
        # Tick spacing
        ax.flat[idx].set_ylim([-4, 4])
        # yticks
        locs = ax.flat[idx].get_yticks()
        loc = plticker.FixedLocator(locs)
        ax.flat[i].yaxis.set_major_locator(loc)
        ax.flat[i].set_ylabel('')
        ax.flat[idx].set_yticklabels([])
        # xticks
        ax.flat[idx].set_xlim([0.7, 1.3])
        ax.flat[i].xaxis.set_major_formatter(StrMethodFormatter('{x:,.1f}'))
        ax.flat[idx].tick_params(axis='x', colors='white')
        if not i%11 or i == shift:
            ax.flat[i].set_ylabel('SPEI', color='white')
        if not i%11 or i == shift:
            ax.flat[i].set_yticklabels([None, -2, 0, 2, None], color='white')
            
    # Legend stuff
    if nofac:
        patches = [mpatches.Patch(facecolor=c0, edgecolor='k', alpha=alpha),
                   mpatches.Patch(facecolor=c1, edgecolor='k', alpha=alpha),
                   mpatches.Patch(facecolor=c2, edgecolor='k', alpha=alpha)]
        labels = ['W.o. glaciers',
                  'W. glaciers',
                  'W. glaciers, nf']
    else:
        patches = [mpatches.Patch(facecolor=c0, edgecolor='k', alpha=alpha),
                   mpatches.Patch(facecolor=c1, edgecolor='k', alpha=alpha)]
        labels = ['W.o. glaciers',
                  'W. glaciers']
    fig.legend(patches, labels, bbox_to_anchor=(0.23, 0.87),
               frameon=True, framealpha=1,  labelcolor='k',
               fontsize=12)
    # Make the first five plots invisible.
    for i in range(shift):
        ax.flat[i].set_visible(False)
        #plt.title(f'SPEI climatology {basin.RIVER_BASI}');
    plt.subplots_adjust(wspace=0.1, hspace=0.3)
    
    fig.patch.set_facecolor('none')
    
    if savefig:
        fig_name = f'spei_clim_{rcp}_{season}.pdf'
        path = os.path.join(fig_path, fig_name)
        fig.savefig(path, bbox_inches='tight')

In [ ]:
plot_spei_climatology('rcp60', savefig=True)

In [ ]:
plot_spei_climatology('rcp45', savefig=True)

In [ ]:
basins_df.iloc[0].lat > 0

In [ ]:
plot_spei_climatology('rcp45', season='summer', savefig=True)

## Paper version of climatology plot

In [ ]:
def plot_spei_climatology_paper(ssp, season=None, savefig=False, lat_sort=False):
    '''Plot the SPEI climatology (violins) of all basins for a certain rcp
    scenario. Paper version.
    
    Args:
    -----
    rcp: str
        I.e. 'rcp26'
    season: str
        Choose a season to plot. 'winter' or 'summer'.
    '''
    selection_dict = {'summer': {True: 'JJA', False: 'DJF'},
                      'winter': {True: 'DJF', False: 'JJA'}}
    
    # font size
    font_size = 7
    # Create the figure
    fig, ax = plt.subplots(ncols=7, nrows=11, figsize=(15/2.54, 24/2.54),
                           sharex=True, sharey=False)
    c0 = 'C1'
    c1 = 'C0'
    c2 = 'C2'
    alpha = 0.5
    basins = basins_df
    if lat_sort:
        basins['lat'] = basins['geometry'].centroid.y
        basins = basins.sort_values('lat', ascending=False)
        
    for i, basin in enumerate(basins.MRBID):
        shift = 2
        idx = i + shift
        mrbid = str(basin)
        # Get the ensembles
        spei_ens, spei_adj_ens = get_spei_ensemble(mrbid, ssp)
        
        if season is not None:
            # Check wether we are in NH (true) or SH (False)
            hemisphere = basins.iloc[i]['lat'] > 0
            # Select the season.
            selection = selection_dict[season][hemisphere]
            spei = spei.sel(time=(spei['time.season']==selection))
            spei_adj = spei_adj.sel(time=(spei_adj['time.season']==selection))
            


        # spei_ens = spei_ens.sel(time=spei_ens['time.season'] == 'JJA')
        # spei_adj_ens = spei_adj_ens.sel(time=spei_adj_ens['time.season'] == 'JJA')

        lw = 0.5
        # Add the mean
        v1 = ax.flat[idx].violinplot(spei_ens.mean(dim='gcm'), showextrema=False)
        v2 = ax.flat[idx].violinplot(spei_adj_ens.mean(dim='gcm'), showextrema=False)

        for pc in v1['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            # pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor('C1')
            pc.set_edgecolor('k')
            pc.set_alpha(0.5)
            pc.set_linewidth(lw)
            pc.set_zorder(2)
        for pc in v2['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            # pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor('C0')
            pc.set_edgecolor('k')
            pc.set_alpha(0.5)
            pc.set_linewidth(lw)
            pc.set_zorder(2)

        # Add the 25th quantile 
        v3 = ax.flat[idx].violinplot(spei_ens.quantile(q=0.25, dim='gcm'), showextrema=False)
        v4 = ax.flat[idx].violinplot(spei_adj_ens.quantile(q=0.25, dim='gcm'), showextrema=False)
        for pc in v3['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], -np.inf, m)
            pc.set_facecolor('none')
            pc.set_edgecolor('C1')
            pc.set_linewidth(lw)
            pc.set_alpha(1)
        for pc in v4['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], -np.inf, m)
            pc.set_facecolor('none')
            pc.set_edgecolor('C0')
            pc.set_linewidth(lw)
            pc.set_alpha(1)

        # Add the 75th quantile 
        v3 = ax.flat[idx].violinplot(spei_ens.quantile(q=0.75, dim='gcm'), showextrema=False)
        v4 = ax.flat[idx].violinplot(spei_adj_ens.quantile(q=0.75, dim='gcm'), showextrema=False)
        for pc in v3['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor('none')
            pc.set_edgecolor('C1')
            pc.set_linewidth(lw)
            pc.set_alpha(1)
        for pc in v4['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor('none')
            pc.set_edgecolor('C0')
            pc.set_linewidth(lw)
            pc.set_alpha(1)

        # Get the basin name. 
        name = basins.iloc[i].RIVER_BASI
        # We don't need to plot the alternative names for now, so split it.
        name = re.split(r'[\(\)]', name)[0]
        if mrbid == '2910':
            name = name.split()[1]
        if mrbid == '6101':
            name = name.split()[0]
        ax.flat[idx].set_title(f'{name}', fontsize=font_size, pad=2)
        # Tick spacing
        #ax.flat[idx].set_ylim([-4, 4])
        # yticks
        # locs = ax.flat[idx].get_yticks()
        # loc = plticker.FixedLocator(locs)
        # ax.flat[i].yaxis.set_major_locator(loc)
        ax.flat[i].set_ylabel('')
        ax.flat[idx].set_yticklabels([])
        # xticks
        # ax.flat[idx].set_xlim([0.7, 1.3])
        # ax.flat[idx].set_xticks([0.85, 1.0, 1.15])
        # ax.flat[i].xaxis.set_major_formatter(StrMethodFormatter('{x:,.2f}'))
        ax.flat[idx].tick_params(axis='x', labelsize=font_size)
        ax.flat[idx].set_xticklabels([])
        # Add axis info to the first plot
        if i == shift:
            ax.flat[i].set_ylabel('SPEI', fontsize=font_size, labelpad=1)
            # ax.flat[i].set_yticklabels([None, -2, 0, 2, None], fontsize=font_size)
            ax.flat[i].tick_params('y', pad=-4)
            
        ax.flat[idx].grid(which='both', linewidth=0.5)
    # Legend stuff
    patches = [mpatches.Patch(facecolor=c0, edgecolor='k', alpha=alpha),
               mpatches.Patch(facecolor=c1, edgecolor='k', alpha=alpha)]
    labels = ['W.o. glaciers',
              'W. glaciers']
    #  Add the legend.
    fig.legend(patches, labels, bbox_to_anchor=(0.30, 0.895),
               frameon=True, framealpha=1,  labelcolor='k',
               fontsize=font_size, title=f'SPEI distributions\nfor {ssp}', title_fontsize=font_size)
    
    # Make the first plots invisible.
    for i in range(shift):
        ax.flat[i].set_visible(False)
        #plt.title(f'SPEI climatology {basin.RIVER_BASI}');
    plt.subplots_adjust(wspace=0.1, hspace=0.3)
    
    
    if savefig:
        fig_name = f'spei_clim_{ssp}_{season}_paper.pdf'
        path = os.path.join(fig_path, fig_name)
        fig.savefig(path, bbox_inches='tight', dpi=120)

In [ ]:
plot_spei_climatology_paper('ssp370', savefig=True)

## Delta SPEI

In [ ]:
basin = basins_df.iloc[24]
mrbid = str(basin.MRBID)

# spei_ens = spei_ens.sel(time=spei_ens['time.season'] == 'JJA')
# spei_adj_ens = spei_adj_ens.sel(time=spei_adj_ens['time.season'] == 'JJA')

font_size = 7
shift = 0.2
fig, ax = plt.subplots(figsize=(5/2.54, 5/2.54))

for i, ssp in enumerate(ssps):
    # Get the ensembles
    spei_ens, spei_adj_ens = get_spei_ensemble(mrbid, ssp)
    # Add the mean
    v1 = ax.violinplot(spei_adj_ens.mean(dim='gcm') - spei_ens.mean(dim='gcm'), showextrema=False, positions=[i*shift])

    for pc in v1['bodies']:
        # get the center
        m = np.mean(pc.get_paths()[0].vertices[:, 0])
        # modify the paths to not go further right than the center
        pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
        pc.set_facecolor('C1')
        pc.set_edgecolor('k')
        pc.set_alpha(0.5)
        pc.set_linewidth(0.5)
        pc.set_zorder(4-(i*0.1))

    # Add the 25th quantile 
    q25_d = spei_adj_ens.quantile(q=0.25, dim='gcm') - spei_ens.quantile(q=0.25, dim='gcm')
    v2 = ax.violinplot(q25_d, showextrema=False, positions=[i*shift])
    for pc in v2['bodies']:
        # get the center
        m = np.mean(pc.get_paths()[0].vertices[:, 0])
        # modify the paths to not go further right than the center
        pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
        pc.set_facecolor('none')
        pc.set_edgecolor('k')
        pc.set_linewidth(0.5)
        pc.set_linestyle(':')
        pc.set_alpha(0.5)

    # Add the 75th quantile 
    q75_d = spei_adj_ens.quantile(q=0.75, dim='gcm') - spei_ens.quantile(q=0.75, dim='gcm')
    v3 = ax.violinplot(q75_d, showextrema=False, positions=[i*shift])
    for pc in v3['bodies']:
        # get the center
        m = np.mean(pc.get_paths()[0].vertices[:, 0])
        # modify the paths to not go further right than the center
        pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
        pc.set_facecolor('none')
        pc.set_edgecolor('k')
        pc.set_linewidth(0.5)
        pc.set_linestyle('--')
        pc.set_alpha(0.5)
# q1, median, q3 = spei.quantile([0.25, 0.5, 0.75]).values
# ax.vlines(ind+offset, q1, q3, color='C0', lw=5)
# q1, median, q3 = spei_adj.quantile([0.25, 0.5, 0.75]).values
# ax.vlines(ind-offset, q1, q3, color='C1', lw=5)

# Fix the xaxis
ax.set_xlim(0, 0.9)
ax.set_xticklabels(ssps, fontdict={'ha':'left', 'rotation': -25})
ax.tick_params('both', labelsize=font_size)
plt.ylabel('$\Delta$-SPEI', fontsize=font_size)
# Legend stuff
patches = [mpatches.Patch(color='C0', alpha=0.5),
           mpatches.Patch(color='C1', alpha=0.5)]
labels = ['W.o. glaciers',
          'W. glaciers']
# plt.legend(patches, labels)
plt.title(f'SPEI climatology {basin.RIVER_BASI}', fontsize=font_size);
plt.savefig(fig_path+'clim_test.pdf', bbox_inches='tight')

## Delta SPEI Europe
### All basins

In [ ]:
'''Plot the SPEI climatology (violins) of all basins for a certain rcp
scenario. Paper version.

Args:
-----
season: str
    Choose a season to plot. 'winter' or 'summer'.
'''
selection_dict = {'summer': {True: 'JJA', False: 'DJF'},
                  'winter': {True: 'DJF', False: 'JJA'}}
lat_sort = False
season = None

basins = basins_df.loc[basins_df.CONTINENT == 'Europe']

shift = 1
nbasins = len(basins)
ncols = 4
nrows = (nbasins // ncols) + shift
nplots = ncols*nrows
height_factor = 24/6
height = height_factor * nrows

# Create the figure
fig, ax = plt.subplots(ncols=ncols, nrows=nrows, figsize=(15/2.54, height/2.54),
                       sharex=True, sharey=False)
colors = ['C0', 'C1', 'C2', 'C3']
# font size
font_size = 7
c0 = 'C1'
c1 = 'C0'
c2 = 'C2'
alpha = 0.5
if lat_sort:
    basins['lat'] = basins['geometry'].centroid.y
    basins = basins.sort_values('lat', ascending=False)

for i, basin in enumerate(basins.MRBID):
    idx = i + shift
    mrbid = str(basin)

    if season is not None:
        # Check wether we are in NH (true) or SH (False)
        hemisphere = basins.iloc[i]['lat'] > 0
        # Select the season.
        selection = selection_dict[season][hemisphere]
        spei = spei.sel(time=(spei['time.season']==selection))
        spei_adj = spei_adj.sel(time=(spei_adj['time.season']==selection))




    lw = 0.5
    dist_shift = 0.2
    # Add the mean
    for j, ssp in enumerate(ssps):
        # Get the ensembles
        spei_ens, spei_adj_ens = get_spei_ensemble(mrbid, ssp)
        # Add the mean
        v1 = ax.flat[idx].violinplot(spei_adj_ens.mean(dim='gcm') - spei_ens.mean(dim='gcm'),
                                     showextrema=False, positions=[0.01+j*dist_shift])

        for pc in v1['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor(colors[j])
            pc.set_edgecolor('k')
            pc.set_alpha(0.5)
            pc.set_linewidth(0.5)
            pc.set_zorder(4-(j*0.1))

        # Add the 25th quantile 
        q25_d = spei_adj_ens.quantile(q=0.25, dim='gcm') - spei_ens.quantile(q=0.25, dim='gcm')
        v2 = ax.flat[idx].violinplot(q25_d, showextrema=False, positions=[j*dist_shift])
        for pc in v2['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor('none')
            pc.set_edgecolor('k')
            pc.set_linewidth(0.5)
            pc.set_linestyle(':')
            pc.set_alpha(0.5)

        # Add the 75th quantile 
        q75_d = spei_adj_ens.quantile(q=0.75, dim='gcm') - spei_ens.quantile(q=0.75, dim='gcm')
        v3 = ax.flat[idx].violinplot(q75_d, showextrema=False, positions=[j*dist_shift])
        for pc in v3['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor('none')
            pc.set_edgecolor('k')
            pc.set_linewidth(0.5)
            pc.set_linestyle('--')
            pc.set_alpha(0.5)

    # Get the basin name. 
    name = basins.iloc[i].RIVER_BASI
    # We don't need to plot the alternative names for now, so split it.
    name = re.split(r'[\(\)]', name)[0]
    if mrbid == '2910':
        name = name.split()[1]
    if mrbid == '6101':
        name = name.split()[0]
    ax.flat[idx].set_title(f'{name}', fontsize=font_size, pad=2)
    # Tick spacing
    # yticks
    ax.flat[i].set_ylabel('')
    ax.flat[idx].ticklabel_format(axis='y', scilimits=(-4, -2), useMathText=True)
    ax.flat[idx].yaxis.offsetText.set_fontsize(5)
    ax.flat[idx].get_yaxis().get_offset_text().set_x(-0.15)
    #if idx != shift:
    # xticks
    ax.flat[idx].set_xlim(0, 0.9)
    ax.flat[idx].set_xticklabels([])
    ax.flat[idx].tick_params(axis='both', labelsize=font_size)
    ax.flat[idx].tick_params('y', pad=-4)

    ax.flat[idx].grid(which='both', linewidth=0.5)
# Legend stuff: Patches
patches = [mpatches.Patch(facecolor=color, edgecolor='k', alpha=alpha, lw=0.6) for color in colors]
labels = [f'GCM mean {ssp}'for ssp in ssps]
# Lines
lines = [Line2D([0], [0], color='k', ls=':', lw=0.6, alpha=0.5), Line2D([0], [0], color='k', ls='--', lw=0.6, alpha=0.5)]
line_labels = ['25th percentile', '75th percentile']

#  Add the legend.
fig.legend(patches + lines, labels + line_labels, bbox_to_anchor=(0.30, 0.895),
           frameon=True, framealpha=1,  labelcolor='k',
           fontsize=font_size, title_fontsize=font_size)

# Make the first plots invisible.
for i in range(shift):
    ax.flat[i].set_visible(False)
    
# Margins
plt.subplots_adjust(wspace=0.3, hspace=0.2)
# Labels
fig.text(0.07, 0.5, '$\Delta$-SPEI', va='center', rotation='vertical', fontsize=font_size)
fig.text(0.5, 0.92, '$\Delta$-SPEI distributions in Europe', ha='center', va='center', fontsize=font_size)


fig_name = f'delta_spei_clim_europe_{season}_paper.pdf'
path = os.path.join(fig_path, fig_name)
fig.savefig(path, bbox_inches='tight', dpi=120)

### Selected basins

In [ ]:
'''Plot the SPEI climatology (violins) of all basins for a certain rcp
scenario. Paper version.

Args:
-----
season: str
    Choose a season to plot. 'winter' or 'summer'.
'''
selection_dict = {'summer': {True: 'JJA', False: 'DJF'},
                  'winter': {True: 'DJF', False: 'JJA'}}
lat_sort = False
season = None

basins = basins_df.loc[basins_df.CONTINENT == 'Europe']
# A bit inefficient to do this twise but...
basins_sel = []
for basin in basins.MRBID:
    for ssp in ssps:
        spei_ens, spei_adj_ens = get_spei_ensemble(str(basin), ssp)
        delta_value = (spei_adj_ens.mean(dim='gcm') - spei_ens.mean(dim='gcm')).std()
        if delta_value >= 0.1:
            basins_sel.append(basin)
basins_sel = list(set(basins_sel))
# Select the basins that fit the criteria
basins = basins[basins['MRBID'].isin(basins_sel)]
    

shift = 1
nbasins = len(basins)
ncols = 3
nrows = (nbasins // ncols) + shift
nplots = ncols*nrows
height_factor = 24/6
height = height_factor * nrows

# Create the figure
fig, ax = plt.subplots(ncols=ncols, nrows=nrows, figsize=(15/2.54, height/2.54),
                       sharex=True, sharey=False)
colors = ['C0', 'C1', 'C2', 'C3']
# font size
font_size = 7
c0 = 'C1'
c1 = 'C0'
c2 = 'C2'
alpha = 0.5
if lat_sort:
    basins['lat'] = basins['geometry'].centroid.y
    basins = basins.sort_values('lat', ascending=False)

for i, basin in enumerate(basins.MRBID):
    idx = i + shift
    mrbid = str(basin)

    if season is not None:
        # Check wether we are in NH (true) or SH (False)
        hemisphere = basins.iloc[i]['lat'] > 0
        # Select the season.
        selection = selection_dict[season][hemisphere]
        spei = spei.sel(time=(spei['time.season']==selection))
        spei_adj = spei_adj.sel(time=(spei_adj['time.season']==selection))

    lw = 0.5
    dist_shift = 0.2
    # Add the mean
    for j, ssp in enumerate(ssps):
        # Get the ensembles
        spei_ens, spei_adj_ens = get_spei_ensemble(mrbid, ssp)
        # Add the mean
        v1 = ax.flat[idx].violinplot(spei_adj_ens.mean(dim='gcm') - spei_ens.mean(dim='gcm'),
                                     showextrema=False, positions=[0.01+j*dist_shift])

        for pc in v1['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor(colors[j])
            pc.set_edgecolor('k')
            pc.set_alpha(0.5)
            pc.set_linewidth(0.5)
            pc.set_zorder(4-(j*0.1))

        # Add the 25th quantile 
        q25_d = spei_adj_ens.quantile(q=0.25, dim='gcm') - spei_ens.quantile(q=0.25, dim='gcm')
        v2 = ax.flat[idx].violinplot(q25_d, showextrema=False, positions=[j*dist_shift])
        for pc in v2['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor('none')
            pc.set_edgecolor('k')
            pc.set_linewidth(0.5)
            pc.set_linestyle(':')
            pc.set_alpha(0.5)

        # Add the 75th quantile 
        q75_d = spei_adj_ens.quantile(q=0.75, dim='gcm') - spei_ens.quantile(q=0.75, dim='gcm')
        v3 = ax.flat[idx].violinplot(q75_d, showextrema=False, positions=[j*dist_shift])
        for pc in v3['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor('none')
            pc.set_edgecolor('k')
            pc.set_linewidth(0.5)
            pc.set_linestyle('--')
            pc.set_alpha(0.5)

    # Get the basin name. 
    name = basins.iloc[i].RIVER_BASI
    # We don't need to plot the alternative names for now, so split it.
    name = re.split(r'[\(\)]', name)[0]
    if mrbid == '2910':
        name = name.split()[1]
    if mrbid == '6101':
        name = name.split()[0]
    ax.flat[idx].set_title(f'{name}', fontsize=font_size, pad=2)
    # Tick spacing
    # yticks
    ax.flat[i].set_ylabel('')
    ax.flat[idx].ticklabel_format(axis='y', scilimits=(-4, -2), useMathText=True)
    ax.flat[idx].yaxis.offsetText.set_fontsize(5)
    ax.flat[idx].get_yaxis().get_offset_text().set_x(-0.15)
    #if idx != shift:
    # xticks
    ax.flat[idx].set_xlim(0, 0.9)
    ax.flat[idx].set_xticklabels([])
    ax.flat[idx].tick_params(axis='both', labelsize=font_size)
    ax.flat[idx].tick_params('y', pad=-4)

    ax.flat[idx].grid(which='both', linewidth=0.5)
    
    # Add hlines at 0.
    ax.flat[idx].axhline(0, alpha=0.7, lw=0.5)

# Legend stuff: Patches
patches = [mpatches.Patch(facecolor=color, edgecolor='k', alpha=alpha, lw=0.6) for color in colors]
labels = [f'GCM ens. mean {ssp}'for ssp in ssps]
# Lines
lines = [Line2D([0], [0], color='k', ls=':', lw=0.6, alpha=0.5), Line2D([0], [0], color='k', ls='--', lw=0.6, alpha=0.5)]
line_labels = ['Ens. 25th percentile', 'Ens. 75th percentile']

#  Add the legend.
fig.legend(patches + lines, labels + line_labels, bbox_to_anchor=(0.35, 0.895),
           frameon=True, framealpha=1,  labelcolor='k',
           fontsize=font_size, title_fontsize=font_size)

# Make the first plots invisible.
for i in range(shift):
    ax.flat[i].set_visible(False)
    
# Margins
plt.subplots_adjust(wspace=0.2, hspace=0.2)
# Labels
fig.text(0.07, 0.5, '$\Delta$-SPEI', va='center', rotation='vertical', fontsize=font_size)
fig.text(0.5, 0.93, '$\Delta$-SPEI distributions in Europe where $\sigma \geq 0.1$', ha='center', va='center', fontsize=font_size)


fig_name = f'delta_spei_clim_europe_selection_{season}_paper.pdf'
path = os.path.join(fig_path, fig_name)
fig.savefig(path, bbox_inches='tight', dpi=120)

### Selcted basins presentation

In [ ]:
'''Plot the SPEI climatology (violins) of all basins for a certain rcp
scenario. Paper version.

Args:
-----
season: str
    Choose a season to plot. 'winter' or 'summer'.
'''
selection_dict = {'summer': {True: 'JJA', False: 'DJF'},
                  'winter': {True: 'DJF', False: 'JJA'}}
lat_sort = False
season = None

basins = basins_df.loc[basins_df.CONTINENT == 'Europe']
# A bit inefficient to do this twise but...
basins_sel = []
for basin in basins.MRBID:
    for ssp in ssps:
        spei_ens, spei_adj_ens = get_spei_ensemble(str(basin), ssp)
        delta_value = (spei_adj_ens.mean(dim='gcm') - spei_ens.mean(dim='gcm')).std()
        if delta_value >= 0.1:
            basins_sel.append(basin)
basins_sel = list(set(basins_sel))
# Select the basins that fit the criteria
basins = basins[basins['MRBID'].isin(basins_sel)]
    

shift = 1
nbasins = len(basins)
ncols = 3
nrows = (nbasins // ncols) + shift
nplots = ncols*nrows
height_factor = 24/6
height = height_factor * nrows

# Create the figure
fig, ax = plt.subplots(ncols=ncols, nrows=nrows, figsize=(15/2.54, height/2.54),
                       sharex=True, sharey=False)
colors = ['C0', 'C1', 'C2', 'C3']
# font size
font_size = 7
c0 = 'C1'
c1 = 'C0'
c2 = 'C2'
alpha = 0.5
if lat_sort:
    basins['lat'] = basins['geometry'].centroid.y
    basins = basins.sort_values('lat', ascending=False)

for i, basin in enumerate(basins.MRBID):
    idx = i + shift
    mrbid = str(basin)

    if season is not None:
        # Check wether we are in NH (true) or SH (False)
        hemisphere = basins.iloc[i]['lat'] > 0
        # Select the season.
        selection = selection_dict[season][hemisphere]
        spei = spei.sel(time=(spei['time.season']==selection))
        spei_adj = spei_adj.sel(time=(spei_adj['time.season']==selection))

    lw = 0.5
    dist_shift = 0.2
    # Add the mean
    for j, ssp in enumerate(ssps):
        # Get the ensembles
        spei_ens, spei_adj_ens = get_spei_ensemble(mrbid, ssp)
        # Add the mean
        v1 = ax.flat[idx].violinplot(spei_adj_ens.mean(dim='gcm') - spei_ens.mean(dim='gcm'),
                                     showextrema=False, positions=[0.01+j*dist_shift])

        for pc in v1['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor(colors[j])
            pc.set_edgecolor('k')
            pc.set_alpha(0.5)
            pc.set_linewidth(0.5)
            pc.set_zorder(4-(j*0.1))

        # Add the 25th quantile 
        q25_d = spei_adj_ens.quantile(q=0.25, dim='gcm') - spei_ens.quantile(q=0.25, dim='gcm')
        v2 = ax.flat[idx].violinplot(q25_d, showextrema=False, positions=[j*dist_shift])
        for pc in v2['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor('none')
            pc.set_edgecolor('k')
            pc.set_linewidth(0.5)
            pc.set_linestyle(':')
            pc.set_alpha(0.5)

        # Add the 75th quantile 
        q75_d = spei_adj_ens.quantile(q=0.75, dim='gcm') - spei_ens.quantile(q=0.75, dim='gcm')
        v3 = ax.flat[idx].violinplot(q75_d, showextrema=False, positions=[j*dist_shift])
        for pc in v3['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor('none')
            pc.set_edgecolor('k')
            pc.set_linewidth(0.5)
            pc.set_linestyle('--')
            pc.set_alpha(0.5)

    # Get the basin name. 
    name = basins.iloc[i].RIVER_BASI
    # We don't need to plot the alternative names for now, so split it.
    name = re.split(r'[\(\)]', name)[0]
    if mrbid == '2910':
        name = name.split()[1]
    if mrbid == '6101':
        name = name.split()[0]
    ax.flat[idx].set_title(f'{name}', fontsize=font_size, pad=2)
    # Tick spacing
    # yticks
    ax.flat[i].set_ylabel('')
    ax.flat[idx].ticklabel_format(axis='y', scilimits=(-4, -2), useMathText=True)
    ax.flat[idx].yaxis.offsetText.set_fontsize(5)
    ax.flat[idx].get_yaxis().get_offset_text().set_x(-0.15)
    #if idx != shift:
    # xticks
    ax.flat[idx].set_xlim(0, 0.9)
    ax.flat[idx].set_xticklabels([])
    ax.flat[idx].tick_params(axis='both', labelsize=font_size)
    ax.flat[idx].tick_params('y', pad=-4)

    ax.flat[idx].grid(which='both', linewidth=0.5)
    
    # Add hlines at 0.
    ax.flat[idx].axhline(0, alpha=0.7, lw=0.5)

# Legend stuff: Patches
patches = [mpatches.Patch(facecolor=color, edgecolor='k', alpha=alpha, lw=0.6) for color in colors]
labels = [f'GCM ens. mean {ssp}'for ssp in ssps]
# Lines
lines = [Line2D([0], [0], color='k', ls=':', lw=0.6, alpha=0.5), Line2D([0], [0], color='k', ls='--', lw=0.6, alpha=0.5)]
line_labels = ['Ens. 25th percentile', 'Ens. 75th percentile']

#  Add the legend.
fig.legend(patches + lines, labels + line_labels, bbox_to_anchor=(0.35, 0.895),
           frameon=True, framealpha=1,  labelcolor='k',
           fontsize=font_size, title_fontsize=font_size)

# Make the first plots invisible.
for i in range(shift):
    ax.flat[i].set_visible(False)
    
# Margins
plt.subplots_adjust(wspace=0.2, hspace=0.2)
# Labels
fig.text(0.07, 0.5, '$\Delta$-SPEI', va='center', rotation='vertical', fontsize=font_size)
fig.text(0.5, 0.93, '$\Delta$-SPEI distributions in Europe where $\sigma \geq 0.1$', ha='center', va='center', fontsize=font_size)


fig_name = f'delta_spei_clim_europe_selection_{season}_pres.pdf'
path = os.path.join(fig_path, fig_name)
fig.savefig(path, bbox_inches='tight', facecolor='none')

### Histogram test

In [ ]:
'''Plot the SPEI climatology (violins) of all basins for a certain rcp
scenario. Paper version.

Args:
-----
season: str
    Choose a season to plot. 'winter' or 'summer'.
'''
selection_dict = {'summer': {True: 'JJA', False: 'DJF'},
                  'winter': {True: 'DJF', False: 'JJA'}}
lat_sort = False
season = None

basins = basins_df.loc[basins_df.CONTINENT == 'Europe']
# A bit inefficient to do this twise but...
basins_sel = []
for basin in basins.MRBID:
    for ssp in ssps:
        spei_ens, spei_adj_ens = get_spei_ensemble(str(basin), ssp)
        delta_value = (spei_adj_ens.mean(dim='gcm') - spei_ens.mean(dim='gcm')).std()
        if delta_value >= 0.1:
            basins_sel.append(basin)
basins_sel = list(set(basins_sel))
# Select the basins that fit the criteria
basins = basins[basins['MRBID'].isin(basins_sel)]
    

shift = 1
nbasins = len(basins)
ncols = 3
nrows = (nbasins // ncols) + shift
nplots = ncols*nrows
height_factor = 24/6
height = height_factor * nrows

# Create the figure
fig, ax = plt.subplots(ncols=ncols, nrows=nrows, figsize=(15/2.54, height/2.54),
                       sharex=True, sharey=False)
colors = ['C0', 'C1', 'C2', 'C3']
# font size
font_size = 7
c0 = 'C1'
c1 = 'C0'
c2 = 'C2'
alpha = 0.5
if lat_sort:
    basins['lat'] = basins['geometry'].centroid.y
    basins = basins.sort_values('lat', ascending=False)

for i, basin in enumerate(basins.MRBID):
    idx = i + shift
    mrbid = str(basin)

    if season is not None:
        # Check wether we are in NH (true) or SH (False)
        hemisphere = basins.iloc[i]['lat'] > 0
        # Select the season.
        selection = selection_dict[season][hemisphere]
        spei = spei.sel(time=(spei['time.season']==selection))
        spei_adj = spei_adj.sel(time=(spei_adj['time.season']==selection))

    lw = 0.5
    dist_shift = 0.5
    # Add the mean
    for j, ssp in enumerate(ssps):
        # Get the ensembles
        spei_ens, spei_adj_ens = get_spei_ensemble(mrbid, ssp)
        # Add the mean
        _, _, v1 = ax.flat[idx].hist(spei_adj_ens.mean(dim='gcm') - spei_ens.mean(dim='gcm'), bins='fd',
                               orientation='horizontal', bottom=[0.+j*dist_shift], density=True)

        for pc in v1:
            # get the center
            pc.set_facecolor(colors[j])
            #pc.set_edgecolor('k')
            pc.set_alpha(0.5)
            #pc.set_linewidth(0.5)
            pc.set_zorder(4-(j*0.1))
        # Add the 25th quantile 
        q25_d = spei_adj_ens.quantile(q=0.25, dim='gcm') - spei_ens.quantile(q=0.25, dim='gcm')
        #v2 = ax.flat[idx].violinplot(q25_d, showextrema=False, positions=[j*dist_shift])
        # Add the 75th quantile 
        q75_d = spei_adj_ens.quantile(q=0.75, dim='gcm') - spei_ens.quantile(q=0.75, dim='gcm')
        #v3 = ax.flat[idx].violinplot(q75_d, showextrema=False, positions=[j*dist_shift])

    # Get the basin name. 
    name = basins.iloc[i].RIVER_BASI
    # We don't need to plot the alternative names for now, so split it.
    name = re.split(r'[\(\)]', name)[0]
    if mrbid == '2910':
        name = name.split()[1]
    if mrbid == '6101':
        name = name.split()[0]
    ax.flat[idx].set_title(f'{str.titlename}', fontsize=font_size, pad=2)
    # Tick spacing
    # yticks
    ax.flat[i].set_ylabel('')
    #ax.flat[idx].ticklabel_format(axis='y', scilimits=(-4, -2), useMathText=True)
    #ax.flat[idx].yaxis.offsetText.set_fontsize(5)
    #ax.flat[idx].get_yaxis().get_offset_text().set_x(-0.15)
    #if idx != shift:
    # xticks
    ax.flat[idx].set_xlim(0, 2.5)
    #ax.flat[idx].set_xticklabels([])
    ax.flat[idx].tick_params(axis='both', labelsize=font_size)
    ax.flat[idx].tick_params('y', pad=-4)

    ax.flat[idx].grid(which='both', linewidth=0.5)
    
    # Add hlines at 0.
    ax.flat[idx].axhline(0, alpha=0.7, lw=0.5)

# Legend stuff: Patches
patches = [mpatches.Patch(facecolor=color, edgecolor='k', alpha=alpha, lw=0.6) for color in colors]
labels = [f'GCM ens. mean {ssp}'for ssp in ssps]
# Lines
lines = [Line2D([0], [0], color='k', ls=':', lw=0.6, alpha=0.5), Line2D([0], [0], color='k', ls='--', lw=0.6, alpha=0.5)]
line_labels = ['Ens. 25th percentile', 'Ens. 75th percentile']

#  Add the legend.
fig.legend(patches + lines, labels + line_labels, bbox_to_anchor=(0.35, 0.895),
           frameon=True, framealpha=1,  labelcolor='k',
           fontsize=font_size, title_fontsize=font_size)

# Make the first plots invisible.
for i in range(shift):
    ax.flat[i].set_visible(False)
    
# Margins
plt.subplots_adjust(wspace=0.2, hspace=0.2)
# Labels
fig.text(0.07, 0.5, '$\Delta$-SPEI', va='center', rotation='vertical', fontsize=font_size)
fig.text(0.5, 0.93, '$\Delta$-SPEI distributions in Europe where $\sigma \geq 0.1$', ha='center', va='center', fontsize=font_size)


fig_name = f'delta_spei_clim_europe_selection_{season}_paper.pdf'
path = os.path.join(fig_path, fig_name)
#fig.savefig(path, bbox_inches='tight', dpi=120)

## Delta SPEI South America
### All basins

In [ ]:
'''Plot the SPEI climatology (violins) of all basins for a certain rcp
scenario. Paper version.

Args:
-----
season: str
    Choose a season to plot. 'winter' or 'summer'.
'''
selection_dict = {'summer': {True: 'JJA', False: 'DJF'},
                  'winter': {True: 'DJF', False: 'JJA'}}
lat_sort = False
season = None

basins = basins_df.loc[basins_df.CONTINENT == 'South America']

shift = 0
nbasins = len(basins)
ncols = 4
nrows = (nbasins // ncols) + shift
nplots = ncols*nrows
height_factor = 24/6
height = height_factor * nrows

# Create the figure
fig, ax = plt.subplots(ncols=ncols, nrows=nrows, figsize=(15/2.54, height/2.54),
                       sharex=True, sharey=False)
colors = ['C0', 'C1', 'C2', 'C3']
# font size
font_size = 7
c0 = 'C1'
c1 = 'C0'
c2 = 'C2'
alpha = 0.5
if lat_sort:
    basins['lat'] = basins['geometry'].centroid.y
    basins = basins.sort_values('lat', ascending=False)

for i, basin in enumerate(basins.MRBID):
    idx = i + shift
    mrbid = str(basin)

    if season is not None:
        # Check wether we are in NH (true) or SH (False)
        hemisphere = basins.iloc[i]['lat'] > 0
        # Select the season.
        selection = selection_dict[season][hemisphere]
        spei = spei.sel(time=(spei['time.season']==selection))
        spei_adj = spei_adj.sel(time=(spei_adj['time.season']==selection))




    lw = 0.5
    dist_shift = 0.2
    # Add the mean
    for j, ssp in enumerate(ssps):
        # Get the ensembles
        spei_ens, spei_adj_ens = get_spei_ensemble(mrbid, ssp)
        # Add the mean
        v1 = ax.flat[idx].violinplot(spei_adj_ens.mean(dim='gcm') - spei_ens.mean(dim='gcm'),
                                     showextrema=False, positions=[0.01+j*dist_shift])

        for pc in v1['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor(colors[j])
            pc.set_edgecolor('k')
            pc.set_alpha(0.5)
            pc.set_linewidth(0.5)
            pc.set_zorder(4-(j*0.1))

        # Add the 25th quantile 
        q25_d = spei_adj_ens.quantile(q=0.25, dim='gcm') - spei_ens.quantile(q=0.25, dim='gcm')
        v2 = ax.flat[idx].violinplot(q25_d, showextrema=False, positions=[j*dist_shift])
        for pc in v2['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor('none')
            pc.set_edgecolor('k')
            pc.set_linewidth(0.5)
            pc.set_linestyle(':')
            pc.set_alpha(0.5)

        # Add the 75th quantile 
        q75_d = spei_adj_ens.quantile(q=0.75, dim='gcm') - spei_ens.quantile(q=0.75, dim='gcm')
        v3 = ax.flat[idx].violinplot(q75_d, showextrema=False, positions=[j*dist_shift])
        for pc in v3['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor('none')
            pc.set_edgecolor('k')
            pc.set_linewidth(0.5)
            pc.set_linestyle('--')
            pc.set_alpha(0.5)

    # Get the basin name. 
    name = basins.iloc[i].RIVER_BASI
    # We don't need to plot the alternative names for now, so split it.
    name = re.split(r'[\(\)]', name)[0]
    if mrbid == '2910':
        name = name.split()[1]
    if mrbid == '6101':
        name = name.split()[0]
    ax.flat[idx].set_title(f'{name}', fontsize=font_size, pad=2)
    # Tick spacing
    # yticks
    ax.flat[i].set_ylabel('')
    ax.flat[idx].ticklabel_format(axis='y', scilimits=(-4, -2), useMathText=True)
    ax.flat[idx].yaxis.offsetText.set_fontsize(5)
    ax.flat[idx].get_yaxis().get_offset_text().set_x(-0.15)
    #if idx != shift:
    # xticks
    ax.flat[idx].set_xlim(0, 0.9)
    ax.flat[idx].set_xticklabels([])
    ax.flat[idx].tick_params(axis='both', labelsize=font_size)
    ax.flat[idx].tick_params('y', pad=-4)

    ax.flat[idx].grid(which='both', linewidth=0.5)
# Legend stuff: Patches
patches = [mpatches.Patch(facecolor=color, edgecolor='k', alpha=alpha, lw=0.6) for color in colors]
labels = [f'GCM mean {ssp}'for ssp in ssps]
# Lines
lines = [Line2D([0], [0], color='k', ls=':', lw=0.6, alpha=0.5), Line2D([0], [0], color='k', ls='--', lw=0.6, alpha=0.5)]
line_labels = ['25th percentile', '75th percentile']

#  Add the legend.
fig.legend(patches + lines, labels + line_labels, ncol=3, bbox_to_anchor=(0.50, 0.92), loc='center',
           frameon=True, framealpha=1,  labelcolor='k',
           fontsize=font_size, title_fontsize=font_size)

# Make the first plots invisible.
for i in range(shift):
    ax.flat[i].set_visible(False)
    
# Margins
plt.subplots_adjust(wspace=0.3, hspace=0.2)
# Labels
fig.text(0.07, 0.5, '$\Delta$-SPEI', va='center', rotation='vertical', fontsize=font_size)
fig.text(0.5, 0.95, '$\Delta$-SPEI distributions in South America', ha='center', va='center', fontsize=font_size)


fig_name = f'delta_spei_clim_s_america_{season}_paper.pdf'
path = os.path.join(fig_path, fig_name)
fig.savefig(path, bbox_inches='tight', dpi=120)

### Selected basins

In [ ]:
'''Plot the SPEI climatology (violins) of all basins for a certain rcp
scenario. Paper version.

Args:
-----
season: str
    Choose a season to plot. 'winter' or 'summer'.
'''
selection_dict = {'summer': {True: 'JJA', False: 'DJF'},
                  'winter': {True: 'DJF', False: 'JJA'}}
lat_sort = False
season = None

basins = basins_df.loc[(basins_df.CONTINENT == 'South America') | (basins_df.CONTINENT == 'Asia')]
# A bit inefficient to do this twise but...
basins_sel = []
for basin in basins.MRBID:
    for ssp in ssps:
        spei_ens, spei_adj_ens = get_spei_ensemble(str(basin), ssp)
        delta_value = (spei_adj_ens.mean(dim='gcm') - spei_ens.mean(dim='gcm')).std()
        if delta_value >= 0.1:
            basins_sel.append(basin)
basins_sel = list(set(basins_sel))
# Select the basins that fit the criteria
basins = basins[basins['MRBID'].isin(basins_sel)]
    

shift = 0
nbasins = len(basins)
ncols = 3
nrows = (nbasins // ncols) + shift
nplots = ncols*nrows
height_factor = 24/6
height = height_factor * nrows

# Create the figure
fig, ax = plt.subplots(ncols=ncols, nrows=nrows, figsize=(15/2.54, height/2.54),
                       sharex=True, sharey=False)
colors = ['C0', 'C1', 'C2', 'C3']
# font size
font_size = 7
c0 = 'C1'
c1 = 'C0'
c2 = 'C2'
alpha = 0.5
if lat_sort:
    basins['lat'] = basins['geometry'].centroid.y
    basins = basins.sort_values('lat', ascending=False)

for i, basin in enumerate(basins.MRBID):
    idx = i + shift
    mrbid = str(basin)

    if season is not None:
        # Check wether we are in NH (true) or SH (False)
        hemisphere = basins.iloc[i]['lat'] > 0
        # Select the season.
        selection = selection_dict[season][hemisphere]
        spei = spei.sel(time=(spei['time.season']==selection))
        spei_adj = spei_adj.sel(time=(spei_adj['time.season']==selection))

    lw = 0.5
    dist_shift = 0.2
    # Add the mean
    for j, ssp in enumerate(ssps):
        # Get the ensembles
        spei_ens, spei_adj_ens = get_spei_ensemble(mrbid, ssp)
        # Add the mean
        v1 = ax.flat[idx].violinplot(spei_adj_ens.mean(dim='gcm') - spei_ens.mean(dim='gcm'),
                                     showextrema=False, positions=[0.01+j*dist_shift])

        for pc in v1['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor(colors[j])
            pc.set_edgecolor('k')
            pc.set_alpha(0.5)
            pc.set_linewidth(0.5)
            pc.set_zorder(4-(j*0.1))

        # Add the 25th quantile 
        q25_d = spei_adj_ens.quantile(q=0.25, dim='gcm') - spei_ens.quantile(q=0.25, dim='gcm')
        v2 = ax.flat[idx].violinplot(q25_d, showextrema=False, positions=[j*dist_shift])
        for pc in v2['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor('none')
            pc.set_edgecolor('k')
            pc.set_linewidth(0.5)
            pc.set_linestyle(':')
            pc.set_alpha(0.5)

        # Add the 75th quantile 
        q75_d = spei_adj_ens.quantile(q=0.75, dim='gcm') - spei_ens.quantile(q=0.75, dim='gcm')
        v3 = ax.flat[idx].violinplot(q75_d, showextrema=False, positions=[j*dist_shift])
        for pc in v3['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor('none')
            pc.set_edgecolor('k')
            pc.set_linewidth(0.5)
            pc.set_linestyle('--')
            pc.set_alpha(0.5)

    # Get the basin name. 
    name = basins.iloc[i].RIVER_BASI
    # We don't need to plot the alternative names for now, so split it.
    name = re.split(r'[\(\)]', name)[0]
    if mrbid == '2910':
        name = name.split()[1]
    if mrbid == '6101':
        name = name.split()[0]
    ax.flat[idx].set_title(f'{name}', fontsize=font_size, pad=2)
    # Tick spacing
    # yticks
    ax.flat[i].set_ylabel('')
    ax.flat[idx].ticklabel_format(axis='y', scilimits=(-4, -2), useMathText=True)
    ax.flat[idx].yaxis.offsetText.set_fontsize(5)
    ax.flat[idx].get_yaxis().get_offset_text().set_x(-0.15)
    #if idx != shift:
    # xticks
    ax.flat[idx].set_xlim(0, 0.9)
    ax.flat[idx].set_xticklabels([])
    ax.flat[idx].tick_params(axis='both', labelsize=font_size)
    ax.flat[idx].tick_params('y', pad=-4)

    ax.flat[idx].grid(which='both', linewidth=0.5)
    
    # Add hlines at 0.
    ax.flat[idx].axhline(0, alpha=0.7, lw=0.5)

# Legend stuff: Patches
patches = [mpatches.Patch(facecolor=color, edgecolor='k', alpha=alpha, lw=0.6) for color in colors]
labels = [f'GCM ens. mean {ssp}'for ssp in ssps]
# Lines
lines = [Line2D([0], [0], color='k', ls=':', lw=0.6, alpha=0.5), Line2D([0], [0], color='k', ls='--', lw=0.6, alpha=0.5)]
line_labels = ['Ens. 25th percentile', 'Ens. 75th percentile']

#  Add the legend.
fig.legend(patches + lines, labels + line_labels, ncol=3, bbox_to_anchor=(0.50, 0.98), loc='center',
           frameon=True, framealpha=1,  labelcolor='k',
           fontsize=font_size, title_fontsize=font_size)

# Make the first plots invisible.
for i in range(shift):
    ax.flat[i].set_visible(False)
    
# Margins
plt.subplots_adjust(wspace=0.2, hspace=0.2)
# Labels
fig.text(0.07, 0.5, '$\Delta$-SPEI', va='center', rotation='vertical', fontsize=font_size)
fig.text(0.5, 1.07, '$\Delta$-SPEI distributions in Asia and South America where $\sigma \geq 0.1$', ha='center', va='center', fontsize=font_size)


fig_name = f'delta_spei_clim_asia_s_america_selection_{season}_paper.pdf'
path = os.path.join(fig_path, fig_name)
fig.savefig(path, bbox_inches='tight', dpi=120)

### Presentation selected basins

In [ ]:
'''Plot the SPEI climatology (violins) of all basins for a certain rcp
scenario. Paper version.

Args:
-----
season: str
    Choose a season to plot. 'winter' or 'summer'.
'''
selection_dict = {'summer': {True: 'JJA', False: 'DJF'},
                  'winter': {True: 'DJF', False: 'JJA'}}
lat_sort = False
season = None

basins = basins_df.loc[(basins_df.CONTINENT == 'South America') | (basins_df.CONTINENT == 'Asia')]
# A bit inefficient to do this twise but...
basins_sel = []
for basin in basins.MRBID:
    for ssp in ssps:
        spei_ens, spei_adj_ens = get_spei_ensemble(str(basin), ssp)
        delta_value = (spei_adj_ens.mean(dim='gcm') - spei_ens.mean(dim='gcm')).std()
        if delta_value >= 0.1:
            basins_sel.append(basin)
basins_sel = list(set(basins_sel))
# Select the basins that fit the criteria
basins = basins[basins['MRBID'].isin(basins_sel)]
    

shift = 0
nbasins = len(basins)
ncols = 3
nrows = (nbasins // ncols) + shift
nplots = ncols*nrows
height_factor = 24/6
height = height_factor * nrows

# Create the figure
fig, ax = plt.subplots(ncols=ncols, nrows=nrows, figsize=(15/2.54, height/2.54),
                       sharex=True, sharey=False)
colors = ['C0', 'C1', 'C2', 'C3']
# font size
font_size = 7
c0 = 'C1'
c1 = 'C0'
c2 = 'C2'
alpha = 0.5
if lat_sort:
    basins['lat'] = basins['geometry'].centroid.y
    basins = basins.sort_values('lat', ascending=False)

for i, basin in enumerate(basins.MRBID):
    idx = i + shift
    mrbid = str(basin)

    if season is not None:
        # Check wether we are in NH (true) or SH (False)
        hemisphere = basins.iloc[i]['lat'] > 0
        # Select the season.
        selection = selection_dict[season][hemisphere]
        spei = spei.sel(time=(spei['time.season']==selection))
        spei_adj = spei_adj.sel(time=(spei_adj['time.season']==selection))

    lw = 0.5
    dist_shift = 0.2
    # Add the mean
    for j, ssp in enumerate(ssps):
        # Get the ensembles
        spei_ens, spei_adj_ens = get_spei_ensemble(mrbid, ssp)
        # Add the mean
        v1 = ax.flat[idx].violinplot(spei_adj_ens.mean(dim='gcm') - spei_ens.mean(dim='gcm'),
                                     showextrema=False, positions=[0.01+j*dist_shift])

        for pc in v1['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor(colors[j])
            pc.set_edgecolor('k')
            pc.set_alpha(0.5)
            pc.set_linewidth(0.5)
            pc.set_zorder(4-(j*0.1))

        # Add the 25th quantile 
        q25_d = spei_adj_ens.quantile(q=0.25, dim='gcm') - spei_ens.quantile(q=0.25, dim='gcm')
        v2 = ax.flat[idx].violinplot(q25_d, showextrema=False, positions=[j*dist_shift])
        for pc in v2['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor('none')
            pc.set_edgecolor('k')
            pc.set_linewidth(0.5)
            pc.set_linestyle(':')
            pc.set_alpha(0.5)

        # Add the 75th quantile 
        q75_d = spei_adj_ens.quantile(q=0.75, dim='gcm') - spei_ens.quantile(q=0.75, dim='gcm')
        v3 = ax.flat[idx].violinplot(q75_d, showextrema=False, positions=[j*dist_shift])
        for pc in v3['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor('none')
            pc.set_edgecolor('k')
            pc.set_linewidth(0.5)
            pc.set_linestyle('--')
            pc.set_alpha(0.5)

    # Get the basin name. 
    name = basins.iloc[i].RIVER_BASI
    # We don't need to plot the alternative names for now, so split it.
    name = re.split(r'[\(\)]', name)[0]
    if mrbid == '2910':
        name = name.split()[1]
    if mrbid == '6101':
        name = name.split()[0]
    ax.flat[idx].set_title(f'{name}', fontsize=font_size, pad=2)
    # Tick spacing
    # yticks
    ax.flat[i].set_ylabel('')
    ax.flat[idx].ticklabel_format(axis='y', scilimits=(-4, -2), useMathText=True)
    ax.flat[idx].yaxis.offsetText.set_fontsize(5)
    ax.flat[idx].get_yaxis().get_offset_text().set_x(-0.15)
    #if idx != shift:
    # xticks
    ax.flat[idx].set_xlim(0, 0.9)
    ax.flat[idx].set_xticklabels([])
    ax.flat[idx].tick_params(axis='both', labelsize=font_size)
    ax.flat[idx].tick_params('y', pad=-4)

    ax.flat[idx].grid(which='both', linewidth=0.5)
    
    # Add hlines at 0.
    ax.flat[idx].axhline(0, alpha=0.7, lw=0.5)

# Legend stuff: Patches
patches = [mpatches.Patch(facecolor=color, edgecolor='k', alpha=alpha, lw=0.6) for color in colors]
labels = [f'GCM ens. mean {ssp}'for ssp in ssps]
# Lines
lines = [Line2D([0], [0], color='k', ls=':', lw=0.6, alpha=0.5), Line2D([0], [0], color='k', ls='--', lw=0.6, alpha=0.5)]
line_labels = ['Ens. 25th percentile', 'Ens. 75th percentile']

#  Add the legend.
fig.legend(patches + lines, labels + line_labels, ncol=3, bbox_to_anchor=(0.50, 0.98), loc='center',
           frameon=True, framealpha=1,  labelcolor='k',
           fontsize=font_size, title_fontsize=font_size)

# Make the first plots invisible.
for i in range(shift):
    ax.flat[i].set_visible(False)
    
# Margins
plt.subplots_adjust(wspace=0.2, hspace=0.2)
# Labels
fig.text(0.07, 0.5, '$\Delta$-SPEI', va='center', rotation='vertical', fontsize=font_size)
fig.text(0.5, 1.07, '$\Delta$-SPEI distributions in Asia and South America where $\sigma \geq 0.1$', ha='center', va='center', fontsize=font_size)


fig_name = f'delta_spei_clim_asia_s_america_selection_{season}_pres.pdf'
path = os.path.join(fig_path, fig_name)
fig.savefig(path, bbox_inches='tight', facecolor='none')

## Delta SPEI Asia

In [ ]:
'''Plot the SPEI climatology (violins) of all basins for a certain rcp
scenario. Paper version.

Args:
-----
season: str
    Choose a season to plot. 'winter' or 'summer'.
'''
selection_dict = {'summer': {True: 'JJA', False: 'DJF'},
                  'winter': {True: 'DJF', False: 'JJA'}}
lat_sort = False
season = None

basins = basins_df.loc[(basins_df.CONTINENT == 'Asia') | (basins_df.CONTINENT == 'South-West Pacific')]

shift = 0
nbasins = len(basins)
ncols = 4
nrows = (nbasins // ncols) + shift
nplots = ncols*nrows
height_factor = 24/6
height = height_factor * nrows

# Create the figure
fig, ax = plt.subplots(ncols=ncols, nrows=nrows, figsize=(15/2.54, height/2.54),
                       sharex=True, sharey=False)
colors = ['C0', 'C1', 'C2', 'C3']
# font size
font_size = 7
c0 = 'C1'
c1 = 'C0'
c2 = 'C2'
alpha = 0.5
if lat_sort:
    basins['lat'] = basins['geometry'].centroid.y
    basins = basins.sort_values('lat', ascending=False)

for i, basin in enumerate(basins.MRBID):
    idx = i + shift
    mrbid = str(basin)

    if season is not None:
        # Check wether we are in NH (true) or SH (False)
        hemisphere = basins.iloc[i]['lat'] > 0
        # Select the season.
        selection = selection_dict[season][hemisphere]
        spei = spei.sel(time=(spei['time.season']==selection))
        spei_adj = spei_adj.sel(time=(spei_adj['time.season']==selection))




    lw = 0.5
    dist_shift = 0.2
    # Add the mean
    for j, ssp in enumerate(ssps):
        # Get the ensembles
        spei_ens, spei_adj_ens = get_spei_ensemble(mrbid, ssp)
        # Add the mean
        v1 = ax.flat[idx].violinplot(spei_adj_ens.mean(dim='gcm') - spei_ens.mean(dim='gcm'),
                                     showextrema=False, positions=[0.01+j*dist_shift])

        for pc in v1['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor(colors[j])
            pc.set_edgecolor('k')
            pc.set_alpha(0.5)
            pc.set_linewidth(0.5)
            pc.set_zorder(4-(j*0.1))

        # Add the 25th quantile 
        q25_d = spei_adj_ens.quantile(q=0.25, dim='gcm') - spei_ens.quantile(q=0.25, dim='gcm')
        v2 = ax.flat[idx].violinplot(q25_d, showextrema=False, positions=[j*dist_shift])
        for pc in v2['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor('none')
            pc.set_edgecolor('k')
            pc.set_linewidth(0.5)
            pc.set_linestyle(':')
            pc.set_alpha(0.5)

        # Add the 75th quantile 
        q75_d = spei_adj_ens.quantile(q=0.75, dim='gcm') - spei_ens.quantile(q=0.75, dim='gcm')
        v3 = ax.flat[idx].violinplot(q75_d, showextrema=False, positions=[j*dist_shift])
        for pc in v3['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor('none')
            pc.set_edgecolor('k')
            pc.set_linewidth(0.5)
            pc.set_linestyle('--')
            pc.set_alpha(0.5)

    # Get the basin name. 
    name = basins.iloc[i].RIVER_BASI
    # We don't need to plot the alternative names for now, so split it.
    name = re.split(r'[\(\)]', name)[0]
    if mrbid == '2910':
        name = name.split()[1]
    if mrbid == '6101':
        name = name.split()[0]
    ax.flat[idx].set_title(f'{name}', fontsize=font_size, pad=2)
    # Tick spacing
    # yticks
    ax.flat[i].set_ylabel('')
    ax.flat[idx].ticklabel_format(axis='y', scilimits=(-4, -2), useMathText=True)
    ax.flat[idx].yaxis.offsetText.set_fontsize(5)
    ax.flat[idx].get_yaxis().get_offset_text().set_x(-0.15)
    #if idx != shift:
    # xticks
    ax.flat[idx].set_xlim(0, 0.9)
    ax.flat[idx].set_xticklabels([])
    ax.flat[idx].tick_params(axis='both', labelsize=font_size)
    ax.flat[idx].tick_params('y', pad=-4)

    ax.flat[idx].grid(which='both', linewidth=0.5)
# Legend stuff: Patches
patches = [mpatches.Patch(facecolor=color, edgecolor='k', alpha=alpha, lw=0.6) for color in colors]
labels = [f'GCM mean {ssp}'for ssp in ssps]
# Lines
lines = [Line2D([0], [0], color='k', ls=':', lw=0.6, alpha=0.5), Line2D([0], [0], color='k', ls='--', lw=0.6, alpha=0.5)]
line_labels = ['25th percentile', '75th percentile']

#  Add the legend.
fig.legend(patches + lines, labels + line_labels, ncol=3, bbox_to_anchor=(0.50, 0.92), loc='center',
           frameon=True, framealpha=1,  labelcolor='k',
           fontsize=font_size, title_fontsize=font_size)

# Make the first plots invisible.
for i in range(shift):
    ax.flat[i].set_visible(False)
    
# Margins
plt.subplots_adjust(wspace=0.3, hspace=0.2)
# Labels
fig.text(0.07, 0.5, '$\Delta$-SPEI', va='center', rotation='vertical', fontsize=font_size)
fig.text(0.5, 0.95, '$\Delta$-SPEI distributions in Asia and New Zealand', ha='center', va='center', fontsize=font_size)


fig_name = f'delta_spei_clim_asia_{season}_paper.pdf'
path = os.path.join(fig_path, fig_name)
fig.savefig(path, bbox_inches='tight', dpi=120)

## Delta SPEI North America

In [ ]:
'''Plot the SPEI climatology (violins) of all basins for a certain rcp
scenario. Paper version.

Args:
-----
season: str
    Choose a season to plot. 'winter' or 'summer'.
'''
selection_dict = {'summer': {True: 'JJA', False: 'DJF'},
                  'winter': {True: 'DJF', False: 'JJA'}}
lat_sort = False
season = None

basins = basins_df.loc[(basins_df.CONTINENT == 'North America, Central America and the Caribbean')]

shift = 0
nbasins = len(basins)
ncols = 4
nrows = (nbasins // ncols) + shift
nplots = ncols*nrows
height_factor = 24/6
height = height_factor * nrows

# Create the figure
fig, ax = plt.subplots(ncols=ncols, nrows=nrows, figsize=(15/2.54, height/2.54),
                       sharex=True, sharey=False)
colors = ['C0', 'C1', 'C2', 'C3']
# font size
font_size = 7
c0 = 'C1'
c1 = 'C0'
c2 = 'C2'
alpha = 0.5
if lat_sort:
    basins['lat'] = basins['geometry'].centroid.y
    basins = basins.sort_values('lat', ascending=False)

for i, basin in enumerate(basins.MRBID):
    idx = i + shift
    mrbid = str(basin)

    if season is not None:
        # Check wether we are in NH (true) or SH (False)
        hemisphere = basins.iloc[i]['lat'] > 0
        # Select the season.
        selection = selection_dict[season][hemisphere]
        spei = spei.sel(time=(spei['time.season']==selection))
        spei_adj = spei_adj.sel(time=(spei_adj['time.season']==selection))




    lw = 0.5
    dist_shift = 0.2
    # Add the mean
    for j, ssp in enumerate(ssps):
        # Get the ensembles
        spei_ens, spei_adj_ens = get_spei_ensemble(mrbid, ssp)
        # Add the mean
        v1 = ax.flat[idx].violinplot(spei_adj_ens.mean(dim='gcm') - spei_ens.mean(dim='gcm'),
                                     showextrema=False, positions=[0.01+j*dist_shift])

        for pc in v1['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor(colors[j])
            pc.set_edgecolor('k')
            pc.set_alpha(0.5)
            pc.set_linewidth(0.5)
            pc.set_zorder(4-(j*0.1))

        # Add the 25th quantile 
        q25_d = spei_adj_ens.quantile(q=0.25, dim='gcm') - spei_ens.quantile(q=0.25, dim='gcm')
        v2 = ax.flat[idx].violinplot(q25_d, showextrema=False, positions=[j*dist_shift])
        for pc in v2['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor('none')
            pc.set_edgecolor('k')
            pc.set_linewidth(0.5)
            pc.set_linestyle(':')
            pc.set_alpha(0.5)

        # Add the 75th quantile 
        q75_d = spei_adj_ens.quantile(q=0.75, dim='gcm') - spei_ens.quantile(q=0.75, dim='gcm')
        v3 = ax.flat[idx].violinplot(q75_d, showextrema=False, positions=[j*dist_shift])
        for pc in v3['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor('none')
            pc.set_edgecolor('k')
            pc.set_linewidth(0.5)
            pc.set_linestyle('--')
            pc.set_alpha(0.5)

    # Get the basin name. 
    name = basins.iloc[i].RIVER_BASI
    # We don't need to plot the alternative names for now, so split it.
    name = re.split(r'[\(\)]', name)[0]
    if mrbid == '2910':
        name = name.split()[1]
    if mrbid == '6101':
        name = name.split()[0]
    ax.flat[idx].set_title(f'{name}', fontsize=font_size, pad=2)
    # Tick spacing
    # yticks
    ax.flat[i].set_ylabel('')
    ax.flat[idx].ticklabel_format(axis='y', scilimits=(-4, -2), useMathText=True)
    ax.flat[idx].yaxis.offsetText.set_fontsize(5)
    ax.flat[idx].get_yaxis().get_offset_text().set_x(-0.15)
    #if idx != shift:
    # xticks
    ax.flat[idx].set_xlim(0, 0.9)
    ax.flat[idx].set_xticklabels([])
    ax.flat[idx].tick_params(axis='both', labelsize=font_size)
    ax.flat[idx].tick_params('y', pad=-4)

    ax.flat[idx].grid(which='both', linewidth=0.5)
# Legend stuff: Patches
patches = [mpatches.Patch(facecolor=color, edgecolor='k', alpha=alpha, lw=0.6) for color in colors]
labels = [f'GCM mean {ssp}'for ssp in ssps]
# Lines
lines = [Line2D([0], [0], color='k', ls=':', lw=0.6, alpha=0.5), Line2D([0], [0], color='k', ls='--', lw=0.6, alpha=0.5)]
line_labels = ['25th percentile', '75th percentile']

#  Add the legend.
fig.legend(patches + lines, labels + line_labels, ncol=3, bbox_to_anchor=(0.50, 0.93), loc='center',
           frameon=True, framealpha=1,  labelcolor='k',
           fontsize=font_size, title_fontsize=font_size)

# Make the first plots invisible.
for i in range(shift):
    ax.flat[i].set_visible(False)
    
# Margins
plt.subplots_adjust(wspace=0.3, hspace=0.2)
# Labels
fig.text(0.07, 0.5, '$\Delta$-SPEI', va='center', rotation='vertical', fontsize=font_size)
fig.text(0.5, 0.97, '$\Delta$-SPEI distributions in North America', ha='center', va='center', fontsize=font_size)


fig_name = f'delta_spei_clim_n_america_{season}_paper.pdf'
path = os.path.join(fig_path, fig_name)
fig.savefig(path, bbox_inches='tight', dpi=120)

### Selected basins

In [ ]:
'''Plot the SPEI climatology (violins) of all basins for a certain rcp
scenario. Paper version.

Args:
-----
season: str
    Choose a season to plot. 'winter' or 'summer'.
'''
selection_dict = {'summer': {True: 'JJA', False: 'DJF'},
                  'winter': {True: 'DJF', False: 'JJA'}}
lat_sort = False
season = None

basins = basins_df.loc[(basins_df.CONTINENT == 'North America, Central America and the Caribbean')]
# A bit inefficient to do this twise but...
basins_sel = []
for basin in basins.MRBID:
    for ssp in ssps:
        spei_ens, spei_adj_ens = get_spei_ensemble(str(basin), ssp)
        delta_value = (spei_adj_ens.mean(dim='gcm') - spei_ens.mean(dim='gcm')).std()
        if delta_value >= 0.1:
            basins_sel.append(basin)
basins_sel = list(set(basins_sel))
# Select the basins that fit the criteria
basins = basins[basins['MRBID'].isin(basins_sel)]
    

shift = 2
nbasins = len(basins)
ncols = 3
nrows = (nbasins // ncols) + shift - 1
nplots = ncols*nrows
height_factor = 24/6
height = height_factor * nrows

# Create the figure
fig, ax = plt.subplots(ncols=ncols, nrows=nrows, figsize=(15/2.54, height/2.54),
                       sharex=True, sharey=False)
colors = ['C0', 'C1', 'C2', 'C3']
# font size
font_size = 7
c0 = 'C1'
c1 = 'C0'
c2 = 'C2'
alpha = 0.5
if lat_sort:
    basins['lat'] = basins['geometry'].centroid.y
    basins = basins.sort_values('lat', ascending=False)

for i, basin in enumerate(basins.MRBID):
    idx = i + shift
    mrbid = str(basin)

    if season is not None:
        # Check wether we are in NH (true) or SH (False)
        hemisphere = basins.iloc[i]['lat'] > 0
        # Select the season.
        selection = selection_dict[season][hemisphere]
        spei = spei.sel(time=(spei['time.season']==selection))
        spei_adj = spei_adj.sel(time=(spei_adj['time.season']==selection))

    lw = 0.5
    dist_shift = 0.2
    # Add the mean
    for j, ssp in enumerate(ssps):
        # Get the ensembles
        spei_ens, spei_adj_ens = get_spei_ensemble(mrbid, ssp)
        # Add the mean
        v1 = ax.flat[idx].violinplot(spei_adj_ens.mean(dim='gcm') - spei_ens.mean(dim='gcm'),
                                     showextrema=False, positions=[0.01+j*dist_shift])

        for pc in v1['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor(colors[j])
            pc.set_edgecolor('k')
            pc.set_alpha(0.5)
            pc.set_linewidth(0.5)
            pc.set_zorder(4-(j*0.1))

        # Add the 25th quantile 
        q25_d = spei_adj_ens.quantile(q=0.25, dim='gcm') - spei_ens.quantile(q=0.25, dim='gcm')
        v2 = ax.flat[idx].violinplot(q25_d, showextrema=False, positions=[j*dist_shift])
        for pc in v2['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor('none')
            pc.set_edgecolor('k')
            pc.set_linewidth(0.5)
            pc.set_linestyle(':')
            pc.set_alpha(0.5)

        # Add the 75th quantile 
        q75_d = spei_adj_ens.quantile(q=0.75, dim='gcm') - spei_ens.quantile(q=0.75, dim='gcm')
        v3 = ax.flat[idx].violinplot(q75_d, showextrema=False, positions=[j*dist_shift])
        for pc in v3['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor('none')
            pc.set_edgecolor('k')
            pc.set_linewidth(0.5)
            pc.set_linestyle('--')
            pc.set_alpha(0.5)

    # Get the basin name. 
    name = basins.iloc[i].RIVER_BASI
    # We don't need to plot the alternative names for now, so split it.
    name = re.split(r'[\(\)]', name)[0]
    if mrbid == '2910':
        name = name.split()[1]
    if mrbid == '6101':
        name = name.split()[0]
    ax.flat[idx].set_title(f'{name}', fontsize=font_size, pad=2)
    # Tick spacing
    # yticks
    ax.flat[i].set_ylabel('')
    ax.flat[idx].ticklabel_format(axis='y', scilimits=(-4, -2), useMathText=True)
    ax.flat[idx].yaxis.offsetText.set_fontsize(5)
    ax.flat[idx].get_yaxis().get_offset_text().set_x(-0.15)
    #if idx != shift:
    # xticks
    ax.flat[idx].set_xlim(0, 0.9)
    ax.flat[idx].set_xticklabels([])
    ax.flat[idx].tick_params(axis='both', labelsize=font_size)
    ax.flat[idx].tick_params('y', pad=-4)

    ax.flat[idx].grid(which='both', linewidth=0.5)
    
    # Add hlines at 0.
    ax.flat[idx].axhline(0, alpha=0.7, lw=0.5)

# Legend stuff: Patches
patches = [mpatches.Patch(facecolor=color, edgecolor='k', alpha=alpha, lw=0.6) for color in colors]
labels = [f'GCM ens. mean {ssp}'for ssp in ssps]
# Lines
lines = [Line2D([0], [0], color='k', ls=':', lw=0.6, alpha=0.5), Line2D([0], [0], color='k', ls='--', lw=0.6, alpha=0.5)]
line_labels = ['Ens. 25th percentile', 'Ens. 75th percentile']

#  Add the legend.
fig.legend(patches + lines, labels + line_labels, ncol=2, bbox_to_anchor=(0.35, 0.78), loc='center',
           frameon=True, framealpha=1,  labelcolor='k',
           fontsize=font_size, title_fontsize=font_size)

# Make the first plots invisible.
for i in range(shift):
    ax.flat[i].set_visible(False)
    
# Margins
plt.subplots_adjust(wspace=0.2, hspace=0.2)
# Labels
fig.text(0.07, 0.5, '$\Delta$-SPEI', va='center', rotation='vertical', fontsize=font_size)
fig.text(0.35, 0.86, '$\Delta$-SPEI distributions in North America where $\sigma \geq 0.1$', ha='center', va='center', fontsize=font_size)


fig_name = f'delta_spei_clim_n_america_selection_{season}_paper.pdf'
path = os.path.join(fig_path, fig_name)
fig.savefig(path, bbox_inches='tight', dpi=120)

### Presentation: Selected basins

In [ ]:
'''Plot the SPEI climatology (violins) of all basins for a certain rcp
scenario. Paper version.

Args:
-----
season: str
    Choose a season to plot. 'winter' or 'summer'.
'''
selection_dict = {'summer': {True: 'JJA', False: 'DJF'},
                  'winter': {True: 'DJF', False: 'JJA'}}
lat_sort = False
season = None

basins = basins_df.loc[(basins_df.CONTINENT == 'North America, Central America and the Caribbean')]
# A bit inefficient to do this twise but...
basins_sel = []
for basin in basins.MRBID:
    for ssp in ssps:
        spei_ens, spei_adj_ens = get_spei_ensemble(str(basin), ssp)
        delta_value = (spei_adj_ens.mean(dim='gcm') - spei_ens.mean(dim='gcm')).std()
        if delta_value >= 0.1:
            basins_sel.append(basin)
basins_sel = list(set(basins_sel))
# Select the basins that fit the criteria
basins = basins[basins['MRBID'].isin(basins_sel)]
    

shift = 2
nbasins = len(basins)
ncols = 3
nrows = (nbasins // ncols) + shift - 1
nplots = ncols*nrows
height_factor = 24/6
height = height_factor * nrows

# Create the figure
fig, ax = plt.subplots(ncols=ncols, nrows=nrows, figsize=(15/2.54, height/2.54),
                       sharex=True, sharey=False)
colors = ['C0', 'C1', 'C2', 'C3']
# font size
font_size = 7
c0 = 'C1'
c1 = 'C0'
c2 = 'C2'
alpha = 0.5
if lat_sort:
    basins['lat'] = basins['geometry'].centroid.y
    basins = basins.sort_values('lat', ascending=False)

for i, basin in enumerate(basins.MRBID):
    idx = i + shift
    mrbid = str(basin)

    if season is not None:
        # Check wether we are in NH (true) or SH (False)
        hemisphere = basins.iloc[i]['lat'] > 0
        # Select the season.
        selection = selection_dict[season][hemisphere]
        spei = spei.sel(time=(spei['time.season']==selection))
        spei_adj = spei_adj.sel(time=(spei_adj['time.season']==selection))

    lw = 0.5
    dist_shift = 0.2
    # Add the mean
    for j, ssp in enumerate(ssps):
        # Get the ensembles
        spei_ens, spei_adj_ens = get_spei_ensemble(mrbid, ssp)
        # Add the mean
        v1 = ax.flat[idx].violinplot(spei_adj_ens.mean(dim='gcm') - spei_ens.mean(dim='gcm'),
                                     showextrema=False, positions=[0.01+j*dist_shift])

        for pc in v1['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor(colors[j])
            pc.set_edgecolor('k')
            pc.set_alpha(0.5)
            pc.set_linewidth(0.5)
            pc.set_zorder(4-(j*0.1))

        # Add the 25th quantile 
        q25_d = spei_adj_ens.quantile(q=0.25, dim='gcm') - spei_ens.quantile(q=0.25, dim='gcm')
        v2 = ax.flat[idx].violinplot(q25_d, showextrema=False, positions=[j*dist_shift])
        for pc in v2['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor('none')
            pc.set_edgecolor('k')
            pc.set_linewidth(0.5)
            pc.set_linestyle(':')
            pc.set_alpha(0.5)

        # Add the 75th quantile 
        q75_d = spei_adj_ens.quantile(q=0.75, dim='gcm') - spei_ens.quantile(q=0.75, dim='gcm')
        v3 = ax.flat[idx].violinplot(q75_d, showextrema=False, positions=[j*dist_shift])
        for pc in v3['bodies']:
            # get the center
            m = np.mean(pc.get_paths()[0].vertices[:, 0])
            # modify the paths to not go further right than the center
            pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0], m, np.inf)
            pc.set_facecolor('none')
            pc.set_edgecolor('k')
            pc.set_linewidth(0.5)
            pc.set_linestyle('--')
            pc.set_alpha(0.5)

    # Get the basin name. 
    name = basins.iloc[i].RIVER_BASI
    # We don't need to plot the alternative names for now, so split it.
    name = re.split(r'[\(\)]', name)[0]
    if mrbid == '2910':
        name = name.split()[1]
    if mrbid == '6101':
        name = name.split()[0]
    ax.flat[idx].set_title(f'{name}', fontsize=font_size, pad=2)
    # Tick spacing
    # yticks
    ax.flat[i].set_ylabel('')
    ax.flat[idx].ticklabel_format(axis='y', scilimits=(-4, -2), useMathText=True)
    ax.flat[idx].yaxis.offsetText.set_fontsize(5)
    ax.flat[idx].get_yaxis().get_offset_text().set_x(-0.15)
    #if idx != shift:
    # xticks
    ax.flat[idx].set_xlim(0, 0.9)
    ax.flat[idx].set_xticklabels([])
    ax.flat[idx].tick_params(axis='both', labelsize=font_size)
    ax.flat[idx].tick_params('y', pad=-4)

    ax.flat[idx].grid(which='both', linewidth=0.5)
    
    # Add hlines at 0.
    ax.flat[idx].axhline(0, alpha=0.7, lw=0.5)

# Legend stuff: Patches
patches = [mpatches.Patch(facecolor=color, edgecolor='k', alpha=alpha, lw=0.6) for color in colors]
labels = [f'GCM ens. mean {ssp}'for ssp in ssps]
# Lines
lines = [Line2D([0], [0], color='k', ls=':', lw=0.6, alpha=0.5), Line2D([0], [0], color='k', ls='--', lw=0.6, alpha=0.5)]
line_labels = ['Ens. 25th percentile', 'Ens. 75th percentile']

#  Add the legend.
fig.legend(patches + lines, labels + line_labels, ncol=2, bbox_to_anchor=(0.35, 0.78), loc='center',
           frameon=True, framealpha=1,  labelcolor='k',
           fontsize=font_size, title_fontsize=font_size)

# Make the first plots invisible.
for i in range(shift):
    ax.flat[i].set_visible(False)
    
# Margins
plt.subplots_adjust(wspace=0.2, hspace=0.2)
# Labels
fig.text(0.07, 0.5, '$\Delta$-SPEI', va='center', rotation='vertical', fontsize=font_size)
fig.text(0.35, 0.86, '$\Delta$-SPEI distributions in North America where $\sigma \geq 0.1$', ha='center', va='center', fontsize=font_size)


fig_name = f'delta_spei_clim_n_america_selection_{season}_pres.pdf'
path = os.path.join(fig_path, fig_name)
fig.savefig(path, bbox_inches='tight', facecolor='none')

# Counting droughts

In [ ]:
# A bit inefficient to do this twise but...
basins_sel = []
for basin in basins_df.MRBID:
    for ssp in ssps:
        spei_ens, spei_adj_ens = get_spei_ensemble(str(basin), ssp)
        delta_value = (spei_adj_ens.mean(dim='gcm') - spei_ens.mean(dim='gcm')).std()
        if delta_value >= 0.1:
            basins_sel.append(basin)
basins_sel = list(set(basins_sel))
# Select the basins that fit the criteria
basins = basins_df[basins_df['MRBID'].isin(basins_sel)]

In [ ]:
basins

In [ ]:
def count_droughts(basins_df, threshold=-1.0):
    '''Utility function for counting droughts in the basins depending on
    the inclusion of the glacier runoff or not. Generates a pandas dataframe. Basins as 
    as rows and rcp scenario + glacier runoff inclusion in the columns. Hence it should be
    75 rows and 8 columns.
    
    Args:
    -----
    basins_df: gepandas dataframe
        Contains the basin data.
    threshold: float
        The drought threshold.
    
    Returns:
    --------
    Pandas dataframe with the counts.
    '''
    cases = ['months_', 'months_adj_']
    
    cols = [x + ssp for ssp in ssps for x in cases]
    df = pd.DataFrame(index=basins_df.MRBID, columns=cols)
    for mrbid in basins_df.MRBID:
        for ssp in ssps:
            # Select data and count months.
            spei_ens, spei_adj_ens = get_spei_ensemble(str(mrbid), ssp)
            no_gl_count = (spei_ens.mean(dim='gcm') < threshold).sum()
            gl_count = (spei_adj_ens.mean(dim='gcm') < threshold).sum()
            
            # Add the data to df.
            df.loc[mrbid][f'months_{ssp}'] = float(no_gl_count.values)
            df.loc[mrbid][f'months_adj_{ssp}'] = float(gl_count.values)
    
    return df

In [ ]:
mrbid = basins.MRBID.iloc[11]
ssp = 'ssp245'
spei_ens, spei_adj_ens = get_spei_ensemble(str(mrbid), ssp)
spei_adj_ens.isel(gcm=6).std()

In [ ]:
count_droughts(basins)

In [ ]:
def drought_diff(basins, threshold=-1.0):
    '''Calculate the drought differenct'''
    # Get the counted droughts df
    count_df = count_droughts(basins, threshold)
    # Then we diff the columns, and select every other column.
    # These contain the values we want.
    diff_df = count_df.diff(axis=1).iloc[:, 1::2]
    # Rename the columns in the new df.
    diff_df.columns = ssps
    diff_df = diff_df.abs()
    return diff_df

In [ ]:
drought_diff(basins, threshold=-0.5)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
def plot_drought_diff(basins_df, threshold=-1.0, savefig=False):
    '''Plot the differences in droughts months for all basins in basins_df.
    '''
    # Get the counted droughts df
    drought_count_diff = drought_diff(basins_df)
    # Set up the plot
    rows = basins.shape[0]
    fig, ax = plt.subplots(figsize=(4.5/1.5, rows/1.5))
    for i in range(4):
        for j in range(0, rows):
            # Get the radius for the circle.
            radius = np.interp(drought_count_diff.iloc[j, i],
                               [drought_count_diff.min().min(),
                                drought_count_diff.max().max()],
                               [0.2, 0.4])
            # Get the color.   
            cmap = cm.viridis
            norm = mcolors.Normalize(drought_count_diff.min().min(),
                                     drought_count_diff.max().max())
            col = cmap(norm(drought_count_diff.iloc[j, i]))
            circle = mpatches.CirclePolygon((i, j), color=col, radius=radius,
                                            resolution=30)
            ax.add_artist(circle)
            ax.axis('equal')
    ax.set_xlim(-1.5, 3.5)       
    ax.set_ylim(rows, -0.9)       
    ax.plot([-0.9, 3.5], [rows, -0.9], alpha=0)
    #ax.axis('off')
    
    # Lets get some annotations in here as well. 
    for i in range(rows):
        name = basins.iloc[i].RIVER_BASI
        name = re.split(r'[\(\)]', name)[0]
        name = '-\n'.join(re.findall('.{1,4}', name))
        plt.annotate(xy=(-1.5, i), text=name, size=10,
                     horizontalalignment='left', verticalalignment='center')
    # xlabels...
    fancy_ssps = ['1-2.6', '2-4.5', '3-7.0', '5-8.5']
    for i, rcp in enumerate(fancy_ssps):
        plt.annotate(xy=(i, rows-0.4), text=rcp,
                     horizontalalignment='center', size=10)
    plt.annotate(xy=(-0.8, rows-0.4), text='SSP',
                 horizontalalignment='center', size=10)
    # Colorbar
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('bottom', size='2%', pad=0)
    plt.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap),
                 cax=cax, orientation='horizontal',
                label=f'$\Delta$-months SPEI <{threshold}')
    # Styling
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.grid(False)
    cax.set_facecolor('gray')
    #plt.tight_layout()
    # return diff_df
    if savefig:
        fig_name = f'delta_dots_selection.pdf'
        path = os.path.join(fig_path, fig_name)
        fig.savefig(path, bbox_inches='tight')

In [ ]:
plot_drought_diff(basins) 

# A look at the basins

In [ ]:
# Define the crs
crs = ccrs.EqualEarth()
# Reproject the geometries
new_geometries = [crs.project_geometry(ii, src_crs=ccrs.PlateCarree())
                  for ii in basins['geometry'].values]
# drought_count_diff = drought_diff(basins)
# Main pt
fig, ax = plt.subplots(subplot_kw={'projection': crs})
# Plots dots
for geometry in new_geometries:
    ax.scatter(geometry.centroid.x, geometry.centroid.y)

ax.coastlines(resolution='50m');

In [ ]:
ccrs

In [ ]:
def plot_continent(SPEI_dict, basind_df, continent):
    # Make the selection
    basins = basins_df[basins_df.CONTINENT==continent]
    
    # Select the crs
    extents = {'Europe': (-20, 50, 30, 75),
               'Asia': (50, 180, 0, 90),
               'South America': (-90, -30, 20, -60),
               'North America, Central America and the Caribbean': (-180, -80, 30, 90)
              }
    # Calculate the central longitude.
    central_longitude = extents[continent][0] + abs(extents[continent][1] - extents[continent][0])/2
    crs = ccrs.EqualEarth(central_longitude=central_longitude)
    # Reproject the geometries
    new_geometries = [crs.project_geometry(ii, src_crs=ccrs.PlateCarree())
                      for ii in basins['geometry'].values]

    # Figure setup
    fig, ax = plt.subplots(subplot_kw={'projection': crs}, figsize=(7,7))
    # Add the basins, individual colors.
    # First we need the SPEI difference,
    spei_diff_all = drought_diff(SPEI_dict, basins_df)
    spei_diff = drought_diff(SPEI_dict, basins)
    for i, geometry in enumerate(new_geometries):
        # Get the color.   
        cmap = cm.viridis
        norm = mcolors.Normalize(spei_diff_all.iloc[:, 0].min(), spei_diff_all.iloc[:, 0].max())
        col = cmap(norm(spei_diff.iloc[i, 0]))
        ax.add_geometries(geometry, crs=crs, lw=0.5, color=col, edgecolor='Black')
    ax.coastlines(resolution='50m')
    # Add labels to the basins in a loop.
    # for _, basin in basins.iterrows():
    #     # Get the centroid of the basin.
    #     x = basin.geometry.centroid.x
    #     y = basin.geometry.centroid.y
    #     # Full name of the basin.
    #     name = basin.RIVER_BASI
    #     # We don't need to plot the alternative names for now, so split it.
    #     name = re.split(r'[\(\)]', name)[0]
    #     # Add the text.
    #     ax.text(x, y, name, transform=ccrs.PlateCarree())
    # Set the extent.
    ax.set_extent(extents[continent], crs=ccrs.PlateCarree())

In [ ]:
plot_continent(SPEI_dict, basins_df, 'North America, Central America and the Caribbean')

In [ ]:
plot_continent(SPEI_dict, basins_df, 'Europe')

In [ ]:
plot_continent(SPEI_dict, basins_df, 'Asia')

In [ ]:
plot_continent(SPEI_dict, basins_df, 'South America')

In [ ]:
drought_diff(SPEI_dict, basins_df)

## Scatter plot idea

In [ ]:
spei_diff_all = drought_diff(SPEI_dict, basins_df)
cmap = cm.viridis
norm = mcolors.Normalize(spei_diff_all.iloc[:, 0].min(), spei_diff_all.iloc[:, 0].max())
plt.scatter(basins_df.lat, basins_df.RGI_AREA,
            color=cmap(norm(spei_diff_all.iloc[:, 0].to_numpy(dtype=float))))